# 🏛️ บทที่ 4.1: ถอดรหัสธุรกิจ (Domain Analysis)
---

จากไฟล์เคลมและไฟล์ชดเชยที่ป๋าส่งมา เรามามองข้าม "ตาราง Excel" แล้วมองเป็น "วัตถุในโลกจริง" (Real-world Objects) กันครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Domain Model** | โด-เมน โม-เดล | แบบจำลองของระบบธุรกิจที่เรากำลังจะเขียนโปรแกรม |
| **Value Object** | แวล-ลู ออบ-เจกต์ | ข้อมูลที่เป็นส่วนประกอบ (เช่น จำนวนเงิน, รหัสพัสดุ) ไม่มี ID ของตัวเอง |
| **Entity** | เอน-ติ-ตี | วัตถุหลักที่มีเอกลักษณ์ (ID) ชัดเจน (เช่น ใบเคลม, ใบชดเชยเงิน) |

**🔍 สิ่งที่เราสกัดได้จากไฟล์ของป๋า:**
1. **TrackingNumber (รหัสพัสดุ):** เป็น Value Object ที่ห้ามเป็นค่าว่าง และต้องนำไปจับคู่ได้
2. **Money (จำนวนเงิน):** เป็น Value Object ที่ประกอบด้วย ตัวเลข (Amount) และ สกุลเงิน (Currency)
3. **ClaimTicket (ใบเคลม):** เป็น Entity หลักที่มี `TrackingNumber` เป็นส่วนประกอบ

# 🔴 บทที่ 4.2: กฎข้อแรกของ TDD (Red Phase)
---

เราจะเริ่มจากสิ่งที่เล็กที่สุดคือ **TrackingNumber (รหัสพัสดุ)** ครับ 
ในทางวิศวกรรม เราจะไม่ยอมให้โปรแกรมรับ String มั่วๆ (เช่น `"123"`, `""`, หรือ `None`) เข้ามาในระบบ เราจึงต้องสร้าง Value Object มาคลุมมันไว้ และใช้ `Pydantic` ในการตรวจสอบ (Validate) ความถูกต้องครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Red Phase** | เรด เฟส | ขั้นตอนแรกของ TDD คือการเขียน Test ให้ "พัง" (Fail) ก่อน เพราะเรายังไม่ได้เขียนโค้ดจริง |
| **Validation** | แว-ลิ-เด-ชัน | การตรวจสอบความถูกต้องของข้อมูล |

In [ ]:
import pytest
from pydantic import ValidationError

# สมมติว่าเราเรียกใช้คลาสจากไฟล์ domain (ที่เรากำลังจะสร้าง)
# from domain import TrackingNumber 

# -----------------------------------------
# Test Cases สำหรับ TrackingNumber
# -----------------------------------------

# เทสที่ 1: ใส่ข้อมูลถูกต้อง ต้องทำงานได้
def test_valid_tracking_number():
    # สมมติเราดึงรหัส TH1234567890 มาจากไฟล์ Excel ของป๋า
    tracking = TrackingNumber(value="TH1234567890")
    assert tracking.value == "TH1234567890"

# เทสที่ 2: ใส่ค่าว่าง ต้องพัง (Error)
def test_empty_tracking_number():
    with pytest.raises(ValidationError):
        TrackingNumber(value="") # ห้ามว่างเด็ดขาด

# เทสที่ 3: ใส่ประเภทผิด (เช่น ใส่ตัวเลข) ต้องพัง
def test_invalid_type_tracking_number():
    with pytest.raises(ValidationError):
        TrackingNumber(value=12345678) # ต้องเป็น String เท่านั้น

# 🟢 บทที่ 4.3: ทำให้ Test ผ่าน (Green Phase)
---

พอเรามี Test ดักไว้แล้ว ทีนี้เราจะมาเขียนโค้ด Domain จริงๆ กันครับ เราจะใช้ `Pydantic` ซึ่งเป็นตัวจัดการ Value Object ที่ดีที่สุดใน Python ณ ตอนนี้ครับ

In [ ]:
from pydantic import BaseModel, Field, field_validator

class TrackingNumber(BaseModel):
    """
    Value Object สำหรับจัดการรหัสพัสดุ (Tracking Number)
    """
    # กฎ: ต้องเป็น String, ห้ามว่าง (min_length=1), เว้นวรรคหน้าหลังจะถูกตัดทิ้ง (strip_whitespace)
    value: str = Field(..., min_length=1, json_schema_extra={"strip_whitespace": True})

    # เราสามารถเพิ่มกฎ Business Logic ย่อยๆ ได้ด้วย เช่น รหัสต้องยาวกว่า 3 ตัว
    @field_validator('value')
    def check_length(cls, v):
        if len(v) < 5:
            raise ValueError('Tracking Number สั้นเกินไป ไม่น่าจะใช่ของจริง')
        return v.strip()

# --- ลองใช้งานจริง ---
if __name__ == "__main__":
    # 1. ลองใส่รหัสที่ถูกต้อง (จำลองจากไฟล์ป๋า)
    good_track = TrackingNumber(value="TH1234567890")
    print(f"✅ สำเร็จ: ได้รหัสพัสดุ {good_track.value}")

    # 2. ลองแกล้งใส่รหัสสั้นๆ
    try:
        bad_track = TrackingNumber(value="TH1")
    except Exception as e:
        print(f"❌ โดนดักจับ Error: {e}")

# 💰 บทที่ 4.4: การจัดการเรื่องเงิน (The Money Value Object)
---

ในระบบเคลมสินค้า "เงิน" ไม่ใช่แค่ตัวเลขครับ แต่มันคือ **"มูลค่าที่มีสกุลกำกับ"** เสมอ เราจะไม่ยอมให้ `Amount` ลอยไปลอยมาโดยไม่มี `Currency` ครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Encapsulation** | เอน-แคป-ซู-เล-ชัน | **การห่อหุ้ม** ข้อมูลและพฤติกรรมไว้ด้วยกัน (เช่น เอาเลขกับสกุลเงินมามัดรวมกัน) |
| **Immutability** | อิม-มิว-ทา-บิล-ลิ-ตี | **การไม่เปลี่ยนแปลงค่า** (Value Object เมื่อสร้างแล้วต้องเปลี่ยนค่าไม่ได้ ถ้าจะเปลี่ยนต้องสร้างอันใหม่) |
| **Invariant** | อิน-แว-เรียนต์ | **กฎที่ต้องเป็นจริงเสมอ** (เช่น จำนวนเงินห้ามติดลบ) |
| **Primitive Obsession** | พริ-มิ-ทีฟ ออบ-เซส-ชัน | **การยึดติดกับข้อมูลพื้นฐาน** (อาการที่โปรแกรมเมอร์ชอบใช้ int, float แทนการสร้าง Object เฉพาะทาง) |

**🔍 กฎของ Money ในระบบของป๋า:**
1. **Amount (จำนวนเงิน):** ต้องเป็นตัวเลข และห้ามติดลบ (เพราะนี่คือยอดชดเชย)
2. **Currency (สกุลเงิน):** ต้องเป็นตัวอักษร 3 ตัว (เช่น THB, USD)

In [ ]:
# -----------------------------------------
# Test Cases สำหรับ Money
# -----------------------------------------

def test_valid_money_creation():
    # จำลองค่าจากไฟล์ป๋า: 886.26 THB
    m = Money(amount=886.26, currency="THB")
    assert m.amount == 886.26
    assert m.currency == "THB"

def test_money_negative_amount():
    # กฎคือเงินเคลมต้องไม่ติดลบ ถ้าติดลบต้องพัง (Error)
    with pytest.raises(ValidationError):
        Money(amount=-100.0, currency="THB")

def test_money_invalid_currency_format():
    # สกุลเงินต้องมี 3 ตัวอักษรเท่านั้น (มาตรฐาน ISO)
    with pytest.raises(ValidationError):
        Money(amount=100.0, currency="THAI_BAHT")

In [ ]:
from pydantic import BaseModel, Field, field_validator

class Money(BaseModel):
    """
    Value Object สำหรับจัดการเรื่องเงินและความถูกต้องของสกุลเงิน
    """
    # กฎ: ห้ามติดลบ (ge=0)
    amount: float = Field(..., ge=0) 
    # กฎ: ต้องมี 3 ตัวอักษร (เช่น THB) และทำเป็นตัวพิมพ์ใหญ่เสมอ
    currency: str = Field(..., min_length=3, max_length=3)

    @field_validator('currency')
    def currency_must_be_uppercase(cls, v):
        return v.upper()

    # แถม: พลังพิเศษให้ Money สามารถแสดงผลสวยๆ ได้เอง
    def __str__(self):
        return f"{self.amount:,.2f} {self.currency}"

# --- ลองทดสอบการใช้งาน ---
if __name__ == "__main__":
    try:
        # 1. สร้างเงินจากข้อมูลจริงของป๋า
        claim_money = Money(amount=1477.50, currency="thb")
        print(f"✅ สร้างเงินสำเร็จ: {claim_money}") # ผลลัพธ์จะเป็น 1,477.50 THB (เพราะมี __str__)

        # 2. ลองแกล้งใส่เงินติดลบ
        fake_money = Money(amount=-50, currency="USD")
    except Exception as e:
        print(f"❌ ระบบดักจับเงินติดลบได้: {e}")

💡 ทำไมป๋าต้องทำแบบนี้? (วิศวกรรมซอฟต์แวร์ 101)
ถ้าป๋าใช้แค่ float ในไฟล์ Excel แล้ววันหนึ่งเกิดมีไฟล์จากต่างประเทศส่งเงิน USD มาปน ป๋าอาจจะเผลอเอา 100 (USD) ไปบวกกับ 100 (THB) ได้ผลลัพธ์เป็น 200 ซึ่งผิดมหันต์!

แต่ถ้าป๋าใช้ Money Value Object ป๋าสามารถเขียนโค้ดเพิ่มในคลาสนี้ได้ว่า:

"ถ้าจะเอา Money มาบวกกัน ต้องเช็กก่อนว่า Currency ตรงกันไหม ถ้าไม่ตรงให้ Error ทันที"

นี่คือการป้องกัน Bug ตั้งแต่ระดับ "รากฐาน" ครับป๋า

สรุปสถานะตอนนี้:

ป๋ามี TrackingNumber ไว้จัดการรหัสพัสดุ

ป๋ามี Money ไว้จัดการยอดชดเชย

# 🏛️ บทที่ 4.5: เอนทิตี - สิ่งที่มีตัวตนและตัวตน (Entity & Identity)
---

ในขณะที่ Value Object สนใจแค่ "ค่า" (ถ้าเงิน 100 บาทเท่ากัน ก็คือเหมือนกัน) แต่ **Entity** สนใจเรื่อง **"ตัวตน" (Identity)** ครับ ใบเคลมสองใบอาจจะมียอดเงินเท่ากัน แต่ถ้าเลข Ticket ต่างกัน มันคือคนละใบกันครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Entity** | เอน-ติ-ตี | **วัตถุหลักที่มีเอกลักษณ์** มี ID ประจำตัว และมีการเปลี่ยนแปลงตามกาลเวลา |
| **Identity** | ไอ-เดน-ติ-ตี | **เอกลักษณ์** หรือรหัสที่ใช้ระบุตัวตน (เช่น ticket_id) |
| **Lifecycle** | ไลฟ์-ไซ-เคิล | **วงจรชีวิต** ตั้งแต่ถูกสร้าง, รออนุมัติ, จนถึงจ่ายเงินสำเร็จ |
| **Optimistic Locking** | ออป-ติ-มิส-ติก ล็อก-กิ้ง | **การป้องกันข้อมูลชนกัน** โดยใช้เลขเวอร์ชัน (ป้องกันป๋าสองคนกดบันทึกพร้อมกันแล้วข้อมูลทับกัน) |

**🔍 กฎของ ClaimTicket ในระบบของป๋า:**
1. **ต้องมี ID:** เราจะสร้าง `TicketId` มาเป็นกุญแจหลัก
2. **ต้องมีองค์ประกอบ:** ประกอบด้วย `TrackingNumber` และยอดเงิน `Money`
3. **ต้องมี Version:** สำหรับทำ Optimistic Locking ตามมาตรฐานวิศวกรรมซอฟต์แวร์

📝 ส่วนที่ 2: TDD Phase 1 - RED (เขียนเทสความสัมพันธ์)
ป๋าต้องสร้าง TicketId ขึ้นมาก่อน เพื่อไม่ให้ใช้ str ตรงๆ ตามกฎของเราครับ เพิ่มโค้ดนี้ลงในไฟล์เทสครับ

In [ ]:
# -----------------------------------------
# Test Cases สำหรับ ClaimTicket (Entity)
# -----------------------------------------

def test_claim_ticket_creation():
    # 1. เตรียมชิ้นส่วน (Value Objects)
    tid = TicketId(value="CMP-1001")
    tn = TrackingNumber(value="TH1234567890")
    price = Money(amount=886.26, currency="THB")

# อธิบาย: เหมือนป๋าจะประกอบรถยนต์ ป๋าต้องมี "ล้อ" มี "พวงมาลัย" มี "เครื่องยนต์" ก่อนครับ

# บรรทัดนี้เรากำลังสร้าง Value Objects 3 ตัว เตรียมไว้ในมือ ซึ่งแต่ละตัวมีความฉลาดในตัวเอง (เช่น Money จะรู้ว่าห้ามติดลบ)

    # 2. สร้าง Entity
    ticket = ClaimTicket(
        ticket_id=tid,
        tracking_number=tn,
        compensation_amount=price
    )

# อธิบาย: ป๋าเอาชิ้นส่วนในมือ (tid, tn, price) มาส่งให้คลาส ClaimTicket เพื่อ "ประกอบร่าง" เป็นใบเคลมที่มีตัวตนจริง ๆ (Entity)

# สังเกตว่าเราส่ง Object เข้าไปทั้งก้อน ไม่ได้ส่งแค่ string "CMP-1001" เข้าไปตรง ๆ ตามกฎ No Primitives in Domain ที่ป๋าตั้งไว้ครับ

    # 3. ตรวจสอบความถูกต้อง
    assert ticket.ticket_id.value == "CMP-1001"
    assert ticket.version == 1  # เริ่มต้นต้องเป็นเวอร์ชัน 1

# อธิบาย: ป๋ากำลังสั่งคอมพิวเตอร์ว่า "เฮ้ย! ตรวจดูซิว่า ใบเคลมที่สร้างเสร็จแล้วเนี่ย รหัสข้างในมันยังเป็น CMP-1001 เหมือนเดิมไหม?" ถ้าใช่ เทสนี้จะผ่าน ถ้าไม่ใช่ เทสจะฟ้องว่าพังครับ

# อธิบาย: นี่คือหัวใจของ Optimistic Locking ที่ป๋าเคยบอกจุกทองครับ ป๋ากำลังยืนยันว่า "ใบเคลมที่เพิ่งสร้างใหม่สด ๆ ร้อน ๆ ต้องมีเวอร์ชันเริ่มต้นที่เลข 1 เท่านั้น" เพื่อเอาไว้เช็กในอนาคตว่ามีใครมาแอบแก้ข้อมูลป๋าหรือเปล่า

In [ ]:
from pydantic import BaseModel, Field

# --- ทบทวน: สร้าง TicketId Value Object สั้นๆ ---
class TicketId(BaseModel):
    value: str = Field(..., min_length=1)

# --- นี่คือ Entity หลักของป๋า ---
class ClaimTicket(BaseModel):
    """
    Entity: ใบเคลมสินค้า
    ทำหน้าที่รวบรวมข้อมูลการเคลมและควบคุมกฎธุรกิจ
    """
    ticket_id: TicketId
    tracking_number: TrackingNumber
    compensation_amount: Money
    
    # เพิ่มระบบ Optimistic Locking ตามที่ป๋าต้องการ
    version: int = Field(default=1)

    # Logic ธุรกิจ: การอัปเดตยอดเงินเคลม
    def update_compensation(self, new_money: Money):
        """
        กฎ: การอัปเดตยอดเงิน ต้องเพิ่มเลขเวอร์ชันเสมอ
        """
        self.compensation_amount = new_money
        self.version += 1
        print(f"📢 อัปเดตยอดเงินเป็น {new_money} (Version: {self.version})")

# --- ลองรันการทำงานจริง ---
if __name__ == "__main__":
    # 1. สร้างใบเคลม (Data จากไฟล์ป๋า)
    my_ticket = ClaimTicket(
        ticket_id=TicketId(value="CMP-1001"),
        tracking_number=TrackingNumber(value="TH1234567890"),
        compensation_amount=Money(amount=886.26, currency="THB")
    )

    print(f"📦 ใบเคลมถูกสร้าง: {my_ticket.ticket_id.value}")
    print(f"💰 ยอดเดิม: {my_ticket.compensation_amount}")

    # 2. ลองอัปเดตยอดเงิน (เช่น มีการคำนวณใหม่จากไฟล์ที่ 2)
    new_price = Money(amount=950.00, currency="THB")
    my_ticket.update_compensation(new_price)

# 📦 บทที่ 4.6: แอ็กเกรเกต - การมัดรวมข้อมูล (Aggregate)
---

ในโลกความจริง ข้อมูลมันไม่ได้อยู่อย่างโดดเดี่ยวครับ เหมือนที่ User บอกว่า "สินค้า 1 ชิ้น (Tracking เดียวกัน) อาจมีใบเคลมหลายใบ" 

เราจึงต้องมี **"ตระกร้า"** สำหรับเก็บใบเคลมเหล่านั้นรวมกัน เราเรียกตระกร้านี้ว่า **Aggregate** และเรียกตัวหลักที่คุมตระกร้านี้ว่า **Aggregate Root (หัวหน้ากลุ่ม)** ครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Aggregate** | แอ็ก-เกร-เกต | **กลุ่มของวัตถุ** ที่ต้องอยู่ด้วยกันเพื่อทำงานให้สำเร็จตามกฎธุรกิจ |
| **Aggregate Root** | แอ็ก-เกร-เกต รูท | **ตัวแทนกลุ่ม** เป็นจุดเดียวที่อนุญาตให้คนภายนอกติดต่อด้วย (ในที่นี้คือ Case การเคลม) |
| **Invariance** | อิน-แว-เรียนต์ | **กฎเหล็กที่ต้องรักษา** (เช่น ใน 1 Case ทุกใบเคลมต้องมีเลข Tracking เดียวกัน) |
| **Boundary** | บาวน์-ดารี | **ขอบเขต** สิ่งที่บอกว่าอะไรควรอยู่ในตระกร้านี้ อะไรควรอยู่นอก |

**🔍 กฎของ ClaimCase (Aggregate) ของป๋า:**
1. **ยึด Tracking เป็นหลัก:** ใน 1 Case จะมีได้หลายใบเคลม แต่ทุกใบต้องเป็นของ Tracking นี้เท่านั้น
2. **คุมยอดรวม:** ตระกร้านี้ต้องตอบได้ว่า "รวมแล้ว Case นี้เคลมไปกี่บาท"

In [ ]:
# -----------------------------------------
# Test Cases สำหรับ ClaimCase (Aggregate)
# -----------------------------------------

def test_add_multiple_tickets_to_same_case():
    # 1. เตรียม Tracking เดียวกัน (กุญแจหลัก)
    tn = TrackingNumber(value="TH1234567890")
    
    # 2. สร้าง Aggregate Root (ตระกร้า)
    claim_case = ClaimCase(tracking_number=tn)

    # 3. เตรียมใบเคลม 2 ใบ (จำลองว่าแจ้งซ้ำ 2 ครั้ง)
    ticket1 = ClaimTicket(
        ticket_id=TicketId(value="CMP-1001"),
        tracking_number=tn,
        compensation_amount=Money(amount=500, currency="THB")
    )
    ticket2 = ClaimTicket(
        ticket_id=TicketId(value="CMP-1002"),
        tracking_number=tn,
        compensation_amount=Money(amount=300, currency="THB")
    )

    # 4. ใส่ใบเคลมลงในตระกร้า
    claim_case.add_ticket(ticket1)
    claim_case.add_ticket(ticket2)

    # 5. ตรวจสอบ: ในตระกร้าต้องมี 2 ใบ และยอดรวมต้องเป็น 800 บาท
    assert len(claim_case.tickets) == 2
    assert claim_case.total_compensation.amount == 800.0

In [ ]:
class ClaimCase(BaseModel):
    """
    Aggregate Root: ตระกร้าคุมใบเคลมของสินค้า 1 ชิ้น
    """
    tracking_number: TrackingNumber
    # รายการใบเคลมที่อยู่ในตระกร้านี้ (เริ่มต้นเป็น List ว่าง)
    tickets: list[ClaimTicket] = Field(default_factory=list)
    
    # สถานะเงินรวม (Value Object ที่ป๋าทำไว้)
    # เริ่มต้นที่ 0 THB
    total_compensation: Money = Field(
        default_factory=lambda: Money(amount=0, currency="THB")
    )

    def add_ticket(self, ticket: ClaimTicket):
        """
        กฎธุรกิจ: 
        1. ตรวจสอบว่า Tracking ตรงกันไหม
        2. เพิ่มลง List
        3. อัปเดตยอดเงินรวม
        """
        # เช็กกฎ Invariant: ห้ามเอาใบเคลมคนละสินค้ามาปนกัน
        if ticket.tracking_number.value != self.tracking_number.value:
            raise ValueError("ป๋าครับ! ใบเคลมนี้มันคนละเลข Tracking กันนะ")

        self.tickets.append(ticket)
        
        # อัปเดตยอดรวม (ป๋าเห็นไหมครับ เราเอา Value Object มาคำนวณกันในนี้เลย)
        new_amount = self.total_compensation.amount + ticket.compensation_amount.amount
        self.total_compensation = Money(amount=new_amount, currency="THB")
        
        print(f"✅ เพิ่ม {ticket.ticket_id.value} สำเร็จ! ยอดรวมเคสนี้: {self.total_compensation}")

# --- ลองใช้งานจริง ---
if __name__ == "__main__":
    tn = TrackingNumber(value="TH1234567890")
    my_case = ClaimCase(tracking_number=tn)

    # ลองเพิ่มใบที่ 1
    t1 = ClaimTicket(ticket_id=TicketId(value="CMP-01"), tracking_number=tn, 
                     compensation_amount=Money(amount=100, currency="THB"))
    my_case.add_ticket(t1)

    # ลองเพิ่มใบที่ 2
    t2 = ClaimTicket(ticket_id=TicketId(value="CMP-02"), tracking_number=tn, 
                     compensation_amount=Money(amount=200, currency="THB"))
    my_case.add_ticket(t2)

# 🗄️ บทที่ 4.7: เรโพซิทอรี - ลิ้นชักเก็บข้อมูล (Repository Pattern)
---

ในหลักการ Clean Architecture และ DDD ตัว Domain (กฎธุรกิจที่เราเพิ่งเขียนไป) จะ **ไม่รู้จัก Pandas และไม่รู้จัก Excel เลย** ครับ! มันรู้แค่ว่า "ฉันมีลิ้นชักให้เก็บของนะ ส่วนลิ้นชักนั้นข้างในจะเป็นไม้ หรือเป็นเหล็ก (จะเป็น Excel หรือ SQL) ฉันไม่สน"

การทำแบบนี้เรียกว่าการซ่อนความซับซ้อนครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Repository** | เร-โพ-ซิ-ทอ-รี | **คลังเก็บข้อมูล** ทำหน้าที่บันทึกและดึง Aggregate ออกมา |
| **Abstraction** | แอ็บ-สแตรก-ชัน | **การซ่อนรายละเอียด** ให้ Domain รู้แค่ว่าจะเซฟยังไง แต่ไม่ต้องรู้ว่าโค้ดเซฟลงไฟล์เขียนยังไง |
| **Persistence** | เพอร์-ซิส-เทนซ์ | **การทำให้ข้อมูลคงอยู่** (ไม่หายไปเมื่อปิดโปรแกรม) |
| **In-Memory** | อิน-เมม-โม-รี | **การเก็บในหน่วยความจำ** (ใช้สำหรับทดสอบ TDD ให้รันไวๆ โดยไม่ต้องเปิดไฟล์จริง) |

**🔍 กฎของ Repository ของเดฟป๋า:**
1. **Save:** ต้องรับ `ClaimCase` (Aggregate) ทั้งก้อนไปเซฟได้
2. **Get:** ต้องเอา `TrackingNumber` ไปค้นหา แล้วคืนค่ากลับมาเป็น `ClaimCase` ได้

📝 ส่วนที่ 2: TDD Phase 1 - RED (เขียนเทสเพื่อลองใช้ลิ้นชัก)
เดฟป๋าลองเขียนเทสดูครับ เราจะจำลองการสร้างเคส แล้วจับยัดลงลิ้นชัก จากนั้นลองค้นหาดูว่าลิ้นชักมันคืนของมาให้เราถูกต้องไหม

In [ ]:
# -----------------------------------------
# Test Cases สำหรับ Repository
# -----------------------------------------

def test_repository_can_save_and_retrieve_case():
    # 1. สร้างลิ้นชักจำลอง (In-Memory) สำหรับเทส
    repo = InMemoryClaimRepository()
    
    # 2. สร้างเคสจำลอง
    tn = TrackingNumber(value="TH1234567890")
    claim_case = ClaimCase(tracking_number=tn)
    
    # ลองใส่ใบเคลมเข้าไป 1 ใบ
    ticket = ClaimTicket(
        ticket_id=TicketId(value="CMP-999"),
        tracking_number=tn,
        compensation_amount=Money(amount=1000, currency="THB")
    )
    claim_case.add_ticket(ticket)

    # 3. สั่งเซฟลงลิ้นชัก!
    repo.save(claim_case)

    # 4. สั่งค้นหาจากลิ้นชักด้วยเลข Tracking
    retrieved_case = repo.get_by_tracking(tn)

    # 5. ตรวจสอบว่าหาเจอไหม และข้อมูลครบหรือเปล่า
    assert retrieved_case is not None # ต้องไม่เป็นค่าว่าง
    assert retrieved_case.tracking_number.value == "TH1234567890"
    assert len(retrieved_case.tickets) == 1
    assert retrieved_case.total_compensation.amount == 1000.0

📝 ส่วนที่ 3: TDD Phase 2 - GREEN (เขียนโค้ดลิ้นชักจำลอง)
เราจะสร้าง InMemoryClaimRepository ขึ้นมาเพื่อทำให้เทสผ่านก่อนครับ (ยังไม่เอา Pandas มาใช้นะครับ เพราะหลักการ TDD เราต้องเทสให้ระบบ Logic ผ่านก่อนไปยุ่งกับไฟล์จริง)

In [ ]:
from typing import Optional

# นี่คือ "หน้าตา" ของลิ้นชักที่ระบบเราต้องการ (Interface)
class ClaimRepository:
    def save(self, claim_case: ClaimCase):
        raise NotImplementedError
        
    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        raise NotImplementedError

# นี่คือ "ลิ้นชักของปลอม" ที่เอาไว้ใช้รันเทสให้ไวปรู๊ดปร๊าด (In-Memory)
class InMemoryClaimRepository(ClaimRepository):
    def __init__(self):
        # ใช้ Dictionary ของ Python เป็นฐานข้อมูลจำลองไปก่อน
        self._db = {} 

    def save(self, claim_case: ClaimCase):
        # เอาเลข Tracking เป็น Key และเอา Aggregate ทั้งก้อนเป็น Value
        self._db[claim_case.tracking_number.value] = claim_case
        print(f"💾 [DB จำลอง] บันทึกเคสของพัสดุ {claim_case.tracking_number.value} ลงลิ้นชักเรียบร้อย!")

    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        # ค้นหาใน Dictionary
        found_case = self._db.get(tracking.value)
        if found_case:
            print(f"🔍 [DB จำลอง] ค้นพบเคสของพัสดุ {tracking.value}")
        else:
            print(f"❌ [DB จำลอง] ไม่พบพัสดุ {tracking.value} ในระบบ")
        return found_case

# --- ลองใช้งานจริง ---
if __name__ == "__main__":
    # สร้างลิ้นชัก
    repo = InMemoryClaimRepository()
    
    # สร้างข้อมูลและเซฟ
    tn1 = TrackingNumber(value="TH9999999999")
    case1 = ClaimCase(tracking_number=tn1)
    repo.save(case1)
    
    # ลองค้นหาดู
    result = repo.get_by_tracking(TrackingNumber(value="TH9999999999"))
    print(f"ยอดรวมในระบบตอนนี้: {result.total_compensation}")

# 🧪 บทที่ 4.8.1: อินทิเกรชันเทส - ทดสอบมือหยิบไฟล์ (Integration Test)
---

ในขณะที่เทสก่อนหน้านี้เราทดสอบแค่ "สมอง" (Domain) แต่เทสนี้เราจะทดสอบ **"มือ"** (Adapter) ครับ ว่ามันหยิบของจากไฟล์ Excel แล้วเอามาส่งให้สมองได้ถูกต้องจริงไหม

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Integration Test** | อิน-ทิ-เกร-ชัน เทส | **การทดสอบการเชื่อมต่อ** ระหว่างโปรแกรมของเรากับระบบภายนอก (เช่น ไฟล์ Excel) |
| **Mock File** | ม็อก ไฟล์ | **ไฟล์จำลอง** ที่สร้างขึ้นมาเพื่อใช้ในการรันเทสโดยเฉพาะ |
| **Data Mapping Test** | ดาต้า แมพ-พิง เทส | การตรวจสอบว่าคอลัมน์ใน Excel ถูกแปลงเป็น Object ได้ถูกช่องไหม |
| **Fixtures** | ฟิกซ์-เจอร์ | การเตรียมสภาพแวดล้อมก่อนเริ่มเทส (เช่น การสร้างไฟล์จำลองทิ้งไว้) |

**🔍 สิ่งที่เราจะพิสูจน์ในเทสนี้:**
1. ถ้าส่งไฟล์ Excel ให้ Repository มันต้องอ่านได้โดยไม่ Error
2. ข้อมูลที่อ่านออกมา ต้องถูกแปลงเป็น `ClaimCase` ไม่ใช่เป็นแค่ตาราง Pandas

💻 ส่วนที่ 2: TDD Phase 1 - RED (เขียนเทสให้พังก่อน)
เดฟป๋าเพิ่มโค้ดนี้ลงในไฟล์เทสครับ มันจะพัง (RED) เพราะเรายังไม่ได้เขียนตัว PandasClaimRepository จริงๆ ใน 4.8 ครับ

In [ ]:
import pytest
import pandas as pd
import os

# -----------------------------------------
# Integration Test สำหรับ Pandas Repository
# -----------------------------------------

def test_pandas_repo_can_load_from_real_excel():
    # 1. [Arrange] เตรียมไฟล์ Excel จำลองสำหรับเทส
    test_file = "../Lesson_4/test_integration.xlsx"
    mock_df = pd.DataFrame({
        'complaint_ticket_id': ['TKT-001'],
        'tracking_no': ['TH999999'],
        'compensation_final_amt': [500.0]
    })
    mock_df.to_excel(test_file, index=False)

    try:
        # 2. [Act] ลองใช้ Repository อ่านไฟล์นี้
        repo = PandasClaimRepository(test_file)
        results = repo.get_all_cases()

        # 3. [Assert] ตรวจสอบว่าของที่ได้ออกมา "ไม่ใช่แค่ตาราง" แต่เป็น "Object"
        assert len(results) == 1
        assert isinstance(results[0], ClaimCase) # ต้องเป็น Class ClaimCase
        assert results[0].tracking_number.value == "TH999999"
        assert results[0].total_compensation.amount == 500.0

    finally:
        # ลบไฟล์ทิ้งหลังเทสเสร็จ (รักษาความสะอาด)
        if os.path.exists(test_file):
            os.remove(test_file)

🔍 อธิบายสิ่งที่เทสนี้ตรวจสอบ (Why this test?)
isinstance(results[0], ClaimCase): บรรทัดนี้สำคัญมากครับป๋า! เราไม่ได้ตรวจสอบแค่ว่า "มีข้อมูลไหม" แต่เราตรวจสอบว่า "ข้อมูลนั้นถูกแปลงร่าง (Transform) จากแถวใน Excel มาเป็น Aggregate หรือยัง"

results[0].total_compensation.amount == 500.0: ตรวจสอบว่า amount = row.get(..., 0) ที่ป๋าสงสัยตะกี้เนี่ย มันทำงานได้ถูกต้องจริงไหม ถ้าในไฟล์มี 500 มันต้องได้ 500 ไม่ใช่ได้ 0

# 🔌 บทที่ 4.8: อะแดปเตอร์ - ตัวเชื่อมโลกความจริง (The Adapter Layer)
---

ในสถาปัตยกรรมแบบ Clean Architecture ตัว **Adapter** (อะ-แดป-เตอร์) คือส่วนที่ติดต่อกับภายนอก (เช่น ไฟล์ Excel ของป๋า) แล้วแปลงข้อมูลเหล่านั้นให้กลายเป็นสิ่งที่ Domain เข้าใจครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Adapter Layer** | อะ-แดป-เตอร์ เล-เยอร์ | **ชั้นตัวเชื่อม** ทำหน้าที่แปลงข้อมูลจากภายนอกเข้าสู่ระบบภายใน |
| **Data Transformation** | ดาต้า แทรนส์-ฟอร์-เม-ชัน | **การแปลงรูปข้อมูล** (เช่น แปลงจากแถวใน Excel ให้กลายเป็น Money Object) |
| **Glue Code** | กลู โค้ด | **โค้ดกาว** โค้ดที่ทำหน้าที่เชื่อมส่วนที่ต่างกันเข้าด้วยกัน |
| **Dependency Injection** | ดี-เพน-เดน-ซี อิน-เจก-ชัน | การส่ง "เครื่องมือ" (เช่น ลิ้นชัก) เข้าไปให้ระบบใช้ โดยที่ระบบไม่ต้องสร้างเอง |

**🔍 หน้าที่ของ Pandas ในบทนี้:**
ป๋าจะเห็นว่า Pandas จะไม่ใช่ "พระเอก" อีกต่อไป แต่มันจะเป็น **"คนขับรถขนส่ง"** ที่ไปขนข้อมูลจากไฟล์ Excel มาส่งที่หน้าประตูบ้าน แล้วเราจะใช้โค้ดของเราเปลี่ยนข้อมูลเหล่านั้นให้กลายเป็น `ClaimCase` ครับ

💻 ส่วนที่ 2: โค้ดของจริง! การเอา Pandas มาต่อกับ Domain (Code Cell)
นี่คือจุดที่ป๋าจะเห็นว่าทำไมเราต้องเขียน Value Object ไว้เยอะแยะครับ เพราะตอนเราอ่าน Excel มา เราจะ "ห่อ" (Wrap) ข้อมูลเหล่านั้นทันที

In [ ]:
import pandas as pd

class PandasClaimRepository(ClaimRepository): # สืบทอดมาจากพิมพ์เขียวลิ้นชัก
    def __init__(self, file_path: str):
        self.file_path = file_path

    def get_all_cases(self) -> list[ClaimCase]:
        # 1. ใช้ Pandas อ่านไฟล์ (ทบทวนบทที่ 1)
        df = pd.read_excel(self.file_path)
        
        cases = []
        for index, row in df.iterrows():
            # 2. กระบวนการ "แปลงร่าง" (Transformation)
            # ป๋าดูนะครับ เราเอาข้อมูลดิบจาก row มาห่อด้วย Value Object ทันที!
            tn = TrackingNumber(value=row['tracking_no'])
            tid = TicketId(value=row['complaint_ticket_id'])
            
            # สมมติเราดึงราคามา (ถ้าไม่มีให้เป็น 0)
            amount = row.get('compensation_final_amt', 0)
            money = Money(amount=amount, currency="THB")

            # 3. สร้าง Entity และ Aggregate
            ticket = ClaimTicket(
                ticket_id=tid, 
                tracking_number=tn, 
                compensation_amount=money
            )
            
            # สร้างตระกร้า (Case) แล้วใส่ใบเคลมลงไป
            new_case = ClaimCase(tracking_number=tn)
            new_case.add_ticket(ticket)
            
            cases.append(new_case)
            
        return cases

# --- ลองใช้งานกับไฟล์ของป๋า ---
repo = PandasClaimRepository("../Lesson_4/mock_claim_data.xlsx")
all_cases = repo.get_all_cases()

# 🟢 บทที่ 4.8.2: การทำให้ผ่านเกณฑ์ (Green Phase)
---

ในขั้นตอนนี้ เราจะเขียนโค้ด `PandasClaimRepository` ให้สมบูรณ์ เพื่อให้มั่นใจว่ามันสามารถหยิบไฟล์ Excel ของป๋า แล้วเอามาแปลงเป็น "ตระกร้าเคส" (`ClaimCase`) ได้อย่างถูกต้องตามที่เทสต้องการ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Green Phase** | กรีน เฟส | **ขั้นตอนที่ทำให้เทสผ่าน** โดยเขียนโค้ดให้ทำงานได้ตามเงื่อนไข |
| **Data Mapping** | ดาต้า แมพ-พิง | **การจับคู่ข้อมูล** จากหัวตาราง Excel เข้าสู่ตัวแปรในโปรแกรม |
| **Summation Logic** | ซัม-เม-ชัน ลอ-จิก | **ตรรกะการรวมผล** (เช่น การบวกยอดเงินรวมใน Aggregate) |
| **Robust Code** | โร-บัสต์ โค้ด | **โค้ดที่แข็งแรง** ไม่พังง่ายเมื่อเจอข้อมูลแปลก ๆ |

💻 ส่วนที่ 2: โค้ดตัวเต็ม (เพื่อให้ป๋ารันเทสได้เลย)
เพื่อให้ป๋าเห็นภาพ 100% จุกทองรวมโค้ดตั้งแต่ต้นจนถึงปัจจุบันมาให้ในชุดเดียวครับ ป๋าสามารถก๊อปไปรันในไฟล์เดียวเพื่อดูผล "Test: Passed ✅" ได้เลยครับ

In [ ]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
import pandas as pd
import os

# --- [1] Value Objects (ชิ้นส่วนพื้นฐาน) ---
class TrackingNumber(BaseModel):
    value: str = Field(..., min_length=1)
    @field_validator('value')
    def strip_space(cls, v): return v.strip()

class TicketId(BaseModel):
    value: str = Field(..., min_length=1)

class Money(BaseModel):
    amount: float = Field(..., ge=0)
    currency: str = Field(..., min_length=3, max_length=3)
    def __str__(self): return f"{self.amount:,.2f} {self.currency}"

# --- [2] Entity (ใบแจ้งเคลม) ---
class ClaimTicket(BaseModel):
    ticket_id: TicketId
    tracking_number: TrackingNumber
    compensation_amount: Money
    version: int = 1

# --- [3] Aggregate (แฟ้มเคส - รวมใบแจ้งที่มี Tracking เดียวกัน) ---
class ClaimCase(BaseModel):
    tracking_number: TrackingNumber
    tickets: List[ClaimTicket] = Field(default_factory=list)
    total_compensation: Money = Field(default_factory=lambda: Money(amount=0, currency="THB"))

    def add_ticket(self, ticket: ClaimTicket):
        if ticket.tracking_number.value != self.tracking_number.value:
            raise ValueError("Tracking mismatch")
        self.tickets.append(ticket)
        # คำนวณยอดรวมใหม่ทุกครั้งที่มีใบแจ้งเพิ่มเข้ามา
        new_amt = sum(t.compensation_amount.amount for t in self.tickets)
        self.total_compensation = Money(amount=new_amt, currency="THB")

# --- [4] Repository (ลิ้นชักเหล็ก - ต่อกับ Pandas) ---
class PandasClaimRepository:
    def __init__(self, file_path: str):
        self.file_path = file_path

    def get_all_cases(self) -> List[ClaimCase]:
        # อ่านไฟล์ (รองรับทั้ง CSV และ Excel)
        df = pd.read_csv(self.file_path) if self.file_path.endswith('.csv') else pd.read_excel(self.file_path)
        
        all_cases = {}
        for _, row in df.iterrows():
            # ดึงข้อมูลและห่อด้วย Value Objects
            tn = TrackingNumber(value=str(row['tracking_no']))
            tid = TicketId(value=str(row['complaint_ticket_id']))
            
            # จัดการเรื่องราคา (ถ้าเป็นค่าว่างหรือไม่มี ให้เป็น 0)
            amt = row.get('compensation_final_amt', 0)
            if pd.isna(amt): amt = 0
            
            money = Money(amount=float(amt), currency="THB")

            # สร้าง Entity ใบแจ้งเคลม
            ticket = ClaimTicket(ticket_id=tid, tracking_number=tn, compensation_amount=money)

            # ตรรกะ: ถ้า Tracking นี้ยังไม่มีแฟ้ม ให้สร้างใหม่ ถ้ามีแล้วให้เอาใบแจ้งใส่แฟ้มเดิม
            if tn.value not in all_cases:
                all_cases[tn.value] = ClaimCase(tracking_number=tn)
            
            all_cases[tn.value].add_ticket(ticket)

        return list(all_cases.values())

# --- [5] ส่วนทดสอบ (The Integration Test) ---
if __name__ == "__main__":
    test_file = "test_data.csv"
    # จำลองข้อมูล: มี 1 สินค้า (TH99) แต่แจ้งเคลม 2 รอบ (TKT-01, TKT-02)
    mock_df = pd.DataFrame({
        'complaint_ticket_id': ['TKT-01', 'TKT-02'],
        'tracking_no': ['TH99', 'TH99'], 
        'compensation_final_amt': [100.0, 200.0]
    })
    mock_df.to_csv(test_file, index=False)
    
    try:
        repo = PandasClaimRepository(test_file)
        results = repo.get_all_cases()
        
        print(f"--- 🧪 เริ่มการทดสอบการเชื่อมต่อ ---")
        print(f"พบจำนวนเคสทั้งหมด: {len(results)} เคส")
        
        # ตรวจสอบผลลัพธ์
        assert len(results) == 1, "ควรจะรวมเหลือ 1 เคสเพราะ Tracking เดียวกัน"
        assert results[0].total_compensation.amount == 300.0, "ยอดรวมต้องเป็น 300 (100+200)"
        
        print("✅ ผลการทดสอบ: ผ่านฉลุย (Passed)")
        print(f"💰 ยอดรวมเคส {results[0].tracking_number.value}: {results[0].total_compensation}")
        
    finally:
        if os.path.exists(test_file): os.remove(test_file)

# 🧠 บทที่ 4.10: โดเมนเซอร์วิส - ผู้ประสานงานข้อมูล (Domain Service)
---

บางครั้ง ลอจิกธุรกิจก็ไม่สามารถยัดลงไปใน "ใบเคลม" (Entity) หรือ "แฟ้มเคส" (Aggregate) ตัวใดตัวหนึ่งได้โดยตรง เช่น การไปหยิบยอดเงินจาก "ไฟล์ที่ 2" มาเติมให้ "ไฟล์ที่ 1" 

เราจึงต้องมี **Domain Service** มาทำหน้าที่เป็น "ผู้ช่วย" ในการประสานงานนี้ครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Domain Service** | โด-เมน เซอร์-วิส | **บริการในโดเมน** ใช้สำหรับลอจิกที่ต้องทำงานร่วมกับข้อมูลหลายส่วน |
| **Enrichment** | เอน-ริช-เมนท์ | **การทำให้สมบูรณ์** (การเติมข้อมูลที่ขาดหายไปให้ครบถ้วน) |
| **Dictionary Mapping** | ดิก-ชัน-นะ-รี แมพ-พิง | **การจับคู่ด้วยดิกชันนารี** (เทคนิคที่ไวกว่า VLOOKUP ของ Excel หลายเท่า) |
| **Stateless** | สเตท-เลส | **ความไม่มีสถานะ** (Service ทำหน้าที่แค่รับของมาประมวลผลแล้วคืนของไป ไม่เก็บอะไรไว้กับตัว) |

**🔍 หน้าที่ของ ClaimEnrichmentService:**
1. รับ "แฟ้มเคส" (จากไฟล์ 1) และ "ข้อมูลยอดเงิน" (จากไฟล์ 2) เข้ามา
2. ค้นหาว่า Tracking Number ตรงกันไหม
3. ถ้าเจอ ให้เอาเงินไปอัปเดตในแฟ้มเคสนั้นทันที

📝 ส่วนที่ 2: TDD Phase 1 - RED (เขียนเทสการเติมเงิน)
ป๋าเพิ่มเทสตัวนี้เข้าไปครับ เพื่อพิสูจน์ว่าระบบเรา "แมตช์ของ" เก่งแค่ไหน

In [ ]:
# -----------------------------------------
# Test Case: การเติมยอดเงินชดเชยให้สมบูรณ์
# -----------------------------------------

def test_claim_enrichment_logic():
    # 1. [Arrange] เตรียมเคสที่เงินยังเป็น 0 (จากไฟล์ 1)
    tn = TrackingNumber(value="TH777")
    my_case = ClaimCase(tracking_number=tn)
    # ใส่ใบแจ้งที่เงินเป็น 0 บาทลงไป
    my_case.add_ticket(ClaimTicket(
        ticket_id=TicketId(value="TKT-001"),
        tracking_number=tn,
        compensation_amount=Money(amount=0, currency="THB")
    ))

    # 2. [Arrange] เตรียมข้อมูลเงินจริง (จำลองข้อมูลที่โหลดมาจากไฟล์ 2)
    # เราใช้ Dictionary เพราะมันค้นหาไวกว่า List (Big O เป็น 1)
    money_map = {
        "TH777": Money(amount=1500.0, currency="THB")
    }

    # 3. [Act] เรียกใช้ Service (ที่เรากำลังจะเขียน)
    service = ClaimEnrichmentService()
    service.enrich(my_case, money_map)

    # 4. [Assert] ยอดรวมในเคสต้องเปลี่ยนจาก 0 เป็น 1500
    assert my_case.total_compensation.amount == 1500.0

📝 ส่วนที่ 3: TDD Phase 2 - GREEN (เขียนโค้ด Service จริง)
นี่คือโค้ดที่จะทำให้เทสของป๋าสอบผ่านครับ

In [ ]:
class ClaimEnrichmentService:
    """
    Service สำหรับเติมข้อมูลยอดเงินชดเชยให้สมบูรณ์
    """
    def enrich(self, claim_case: ClaimCase, compensation_map: dict[str, Money]):
        # ดึงเลข Tracking จากแฟ้มเคสออกมา
        tracking_val = claim_case.tracking_number.value
        
        # ค้นหาในข้อมูลไฟล์ 2 (compensation_map)
        if tracking_val in compensation_map:
            real_money = compensation_map[tracking_val]
            
            # อัปเดตเงินในใบแจ้งทุกใบที่อยู่ในเคสนี้
            for ticket in claim_case.tickets:
                ticket.compensation_amount = real_money
            
            # สั่งให้ Aggregate คำนวณยอดรวมใหม่ (Logic อยู่ใน ClaimCase)
            # หมายเหตุ: ในที่นี้เราต้องเรียกเมธอดเพื่ออัปเดตสถานะเงินรวม
            new_amt = sum(t.compensation_amount.amount for t in claim_case.tickets)
            claim_case.total_compensation = Money(amount=new_amt, currency="THB")
            
            print(f"✨ เติมเงินให้ {tracking_val} สำเร็จ: {real_money}")
        else:
            print(f"⚠️ ไม่พบยอดเงินสำหรับ {tracking_val} ในข้อมูลไฟล์ 2")

# 🛡️ บทที่ 4.11: การรับมือกับโลกที่โหดร้าย (Sad Path & Edge Cases)
---

โปรแกรมที่ดีไม่ใช่แค่ทำงานถูกเมื่อข้อมูลถูก แต่ต้อง **"ตายอย่างมีสติ"** หรือ **"รับมือได้"** เมื่อข้อมูลผิดปกติครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Happy Path** | แฮป-ปี้ พาท | **ทางที่ราบรื่น** กรณีที่ User ทำทุกอย่างถูกต้อง ข้อมูลมาครบถ้วน |
| **Sad Path** | แซด พาท | **ทางที่เศร้าหมอง** กรณีที่ข้อมูลผิดพลาดชัดเจน (เช่น ไฟล์เสีย, ใส่ตัวหนังสือในช่องตัวเลข) |
| **Edge Case** | เอจ เคส | **กรณีขอบเขต** กรณีที่ข้อมูล "ถูกตามกฎแต่ไม่ปกติ" (เช่น ยอดเคลม 0.01 บาท หรือ ยอดเคลม 10 ล้านบาท) |
| **Corner Case** | คอร์-เนอร์ เคส | **กรณีมุมอับ** เมื่อมีเงื่อนไข Edge Case หลายอย่างเกิดขึ้นพร้อมกัน (เช่น แจ้งเคลมซ้ำ 1,000 ครั้งใน Tracking เดียว) |
| **Fault Tolerance** | ฟอลต์ ทอ-เลอ-แรนซ์ | **ความทนทานต่อความผิดพลาด** ความสามารถของโปรแกรมที่จะไม่พังวอดวายเมื่อเจอ Error |

**🔍 สิ่งที่เราต้องเพิ่มในระบบของป๋า:**
1. **Sad Path:** ถ้าไฟล์ 2 (เงิน) ไม่มีเลข Tracking ที่อยู่ในไฟล์ 1 เลย โปรแกรมต้องไม่พัง (ต้องข้ามไปได้)
2. **Edge Case:** ถ้ามียอดเงินชดเชยหลายบรรทัดในไฟล์ 2 สำหรับ 1 Tracking เราจะเลือกยอดไหน?

💻 ส่วนที่ 2: เพิ่ม Test Cases สำหรับ Sad Path และ Edge Case
ป๋าลองเพิ่มโค้ดเทสชุดนี้เข้าไปครับ เพื่อดูว่า "สมองกล" ของเราจะจัดการเรื่องยาก ๆ ได้ไหม

In [ ]:
# -----------------------------------------
# Test Cases: Sad Path & Edge Case
# -----------------------------------------

# 1. [Sad Path] กรณีค้นหาไม่เจอ (Lookup Missing)
def test_enrichment_tracking_not_found():
    service = ClaimEnrichmentService()
    tn = TrackingNumber(value="TH-NOT-FOUND")
    case = ClaimCase(tracking_number=tn)
    
    # ข้อมูลไฟล์ 2 ไม่มีเลข TH-NOT-FOUND
    money_map = {"TH-EXIST": Money(amount=100, currency="THB")}
    
    # [Act] ลองเติมเงิน
    service.enrich(case, money_map)
    
    # [Assert] ยอดเงินต้องยังเป็น 0 เหมือนเดิม และโปรแกรมต้องไม่ Error (ไม่ค้าง)
    assert case.total_compensation.amount == 0.0
    print("✅ Sad Path Passed: หาไม่เจอแต่โปรแกรมไม่พัง")

# 2. [Edge Case] กรณีเงินเป็น 0.00 บาท (ยอดน้อยที่สุดที่เป็นไปได้)
def test_money_zero_amount():
    # กฎเราบอกว่า ge=0 (มากกว่าหรือเท่ากับ 0) ดังนั้น 0 ต้องผ่าน
    zero_money = Money(amount=0, currency="THB")
    assert zero_money.amount == 0
    print("✅ Edge Case Passed: รองรับยอดเงิน 0 บาท")

# 3. [Edge Case] กรณีแจ้งเคลมซ้ำจำนวนมาก (Stress Test เบาๆ)
def test_huge_number_of_tickets():
    tn = TrackingNumber(value="TH-BULK")
    case = ClaimCase(tracking_number=tn)
    
    # จำลองการแจ้งซ้ำ 100 ครั้งใน 1 สินค้า
    for i in range(100):
        t = ClaimTicket(
            ticket_id=TicketId(value=f"TKT-{i}"),
            tracking_number=tn,
            compensation_amount=Money(amount=10, currency="THB")
        )
        case.add_ticket(t)
    
    assert len(case.tickets) == 100
    assert case.total_compensation.amount == 1000.0 # 10 * 100
    print("✅ Edge Case Passed: รองรับการแจ้งซ้ำจำนวนมาก")

# 🛡️ บทที่ 4.12: การทดสอบในสภาวะไม่ปกติ (Advanced Testing)
---

ก่อนจะไป GUI เราต้องมั่นใจว่าถ้า User "ส่งของแปลก" เข้ามาในระบบ ระบบเราต้องไม่พัง (Crash) และต้องแจ้งเตือนอย่างสุภาพครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Inconsistent Data** | อิน-คอน-ซิส-เทนท์ ดาต้า | **ข้อมูลที่ไม่สอดคล้องกัน** (เช่น ในไฟล์ระบุเป็นเงิน USD แต่ระบบเราคำนวณแต่ THB) |
| **Data Integrity** | ดาต้า อิน-เทก-ริ-ตี | **ความสมบูรณ์ของข้อมูล** (ความถูกต้องแม่นยำตลอดวงจรชีวิตของข้อมูล) |
| **Type Coercion** | ไทป์ โค-เออ-ชัน | **การบังคับชนิดข้อมูล** (เช่น แปลงคำว่า "100" ใน Excel ให้กลายเป็นตัวเลข 100.0 จริงๆ) |
| **Defensive Programming** | ดี-เฟน-ซีฟ โปรแกรมมิ่ง | **การเขียนโปรแกรมเชิงป้องกัน** (เขียนโค้ดเผื่อไว้เลยว่า User จะทำพัง) |

**🔍 เทสเคสที่เราจะเพิ่ม:**
1. **Sad Path (เงินผิดสกุล):** ถ้าไฟล์ 2 ส่งเงิน USD มา แต่ระบบเราตั้งค่าเป็น THB ไว้ จะต้อง Error ทันที
2. **Edge Case (เว้นวรรคเจ้าปัญหา):** เลข Tracking ใน Excel ชอบมีเว้นวรรคโผล่มา (เช่น `" TH123 "`) ระบบต้องฉลาดพอที่จะ Trim ออกให้เอง
3. **Sad Path (ข้อมูลว่าง/NaN):** ถ้าช่องยอดเงินใน Excel เป็นค่าว่าง (NaN) ระบบต้องมองเป็น 0 ไม่ใช่พัง

💻 ส่วนที่ 2: TDD Phase 1 - RED (เขียนเทสเคสที่โหดขึ้น)
ป๋าเพิ่ม 3 เคสนี้เข้าไปในไฟล์เทสครับ มันจะช่วยดักบั๊กที่เจอได้บ่อยที่สุดในงาน IT Support เลย

In [ ]:
# -----------------------------------------
# Test Cases: Advanced Sad Path & Edge Cases
# -----------------------------------------

# 1. [Sad Path] สกุลเงินไม่ตรงกัน (Currency Mismatch)
def test_money_addition_different_currencies():
    m1 = Money(amount=100, currency="THB")
    m2 = Money(amount=10, currency="USD")
    
    # กฎธุรกิจ: ห้ามเอาเงินคนละสกุลมาบวกกันตรงๆ
    # เราจะแก้โค้ดใน Money ให้ดักเรื่องนี้
    with pytest.raises(ValueError, match="Cannot add different currencies"):
        # สมมติเราเขียนฟังก์ชันบวกเงิน
        m1.add(m2)

# 2. [Edge Case] เลข Tracking มีเว้นวรรค (Whitespace Handling)
def test_tracking_number_trimming():
    # User กดเคาะ Spacebar มาใน Excel "  TH12345  "
    tn = TrackingNumber(value="  TH12345  ")
    
    # ผลลัพธ์ต้องถูกตัดช่องว่างออกอัตโนมัติ (ขอบคุณ Pydantic!)
    assert tn.value == "TH12345"

# 3. [Sad Path] ยอดเงินใน Excel เป็นค่าว่าง (NaN Handling)
def test_repository_handles_nan_amount():
    # จำลองว่า Pandas อ่านเจอค่าว่าง (NaN)
    import numpy as np
    raw_amount = np.nan 
    
    # เราต้องมีลอจิกแปลง NaN เป็น 0 ก่อนส่งให้ Money Object
    processed_amount = 0 if pd.isna(raw_amount) else raw_amount
    money = Money(amount=processed_amount, currency="THB")
    
    assert money.amount == 0.0

💻 ส่วนที่ 3: TDD Phase 2 - GREEN (อัปเกรดโค้ดให้ฉลาดขึ้น)
เราจะไปปรับปรุงคลาส Money และ TrackingNumber ของเราให้รองรับเคสข้างบนครับ

In [ ]:
class Money(BaseModel):
    amount: float = Field(..., ge=0)
    currency: str = Field(..., min_length=3, max_length=3)

    # เพิ่มความสามารถในการ "บวกเงิน" อย่างปลอดภัย
    def add(self, other: 'Money') -> 'Money':
        if self.currency != other.currency:
            raise ValueError(f"Cannot add different currencies: {self.currency} and {other.currency}")
        return Money(amount=self.amount + other.amount, currency=self.currency)

    def __str__(self):
        return f"{self.amount:,.2f} {self.currency}"

class TrackingNumber(BaseModel):
    value: str = Field(..., min_length=1)

    # อัปเกรด Validator ให้ตัดช่องว่าง (Trim) อัตโนมัติ
    @field_validator('value', mode='before')
    @classmethod
    def trim_whitespace(cls, v):
        if isinstance(v, str):
            return v.strip()
        return v

# 🏷️ บทที่ 4.14: เมทาดาต้า และกฎ DRY (Metadata & DRY Principle)
---

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Metadata** | เม-ทา-ดา-ต้า | **ข้อมูลที่ใช้อธิบายข้อมูล** (เช่น บอกว่าคอลัมน์นี้ควรตัดช่องว่างนะ แต่ไม่ได้เป็นคนตัดเอง) |
| **DRY Principle** | ดราย พริน-ซิ-เพิล | **หลักการอย่าเขียนซ้ำ** (Don't Repeat Yourself) |
| **Type Alias** | ไทป์ เอ-เลียส | **การตั้งชื่อเล่นให้ชนิดข้อมูล** (เช่น ตั้งชื่อ 'เงินบาท' แทนที่จะเขียน 'float + THB' ทุกรอบ) |
| **Annotation** | แอน-โน-เท-ชัน | **การระบุคุณลักษณะ** เพื่อให้โปรแกรมเข้าใจวิธีจัดการข้อมูล |

**🔍 ไขข้อข้องใจ `json_schema_extra`:**
ใน Pydantic v2 ตัว `json_schema_extra` มันเป็นแค่ **"ป้ายบอกทาง" (Metadata)** ครับ มันเอาไว้บอกโปรแกรมภายนอก (เช่น Swagger หรือการทำเอกสาร API) ว่าคอลัมน์นี้ควรทำอะไร **แต่ตัวมันเองไม่ได้ลงมือทำครับ!** เหมือนป๋าเขียนป้ายแปะไว้ที่ถังขยะว่า "กรุณาทิ้งลงถัง" แต่ตัวป้ายไม่ได้เดินไปเก็บขยะให้ป๋าครับ (จากที่จุกทองลองรันให้ดูข้างบน ป๋าจะเห็นว่ามันไม่ยอมตัด Space ให้จริง ๆ)

2. วิธีแก้ปัญหา "โค้ดซ้ำ" (The Elegant Way)
เพื่อไม่ให้ป๋าต้องเขียน .strip() หรือ .upper() ในทุก Class เราจะใช้ฟีเจอร์ที่ชื่อว่า Annotated และ StringConstraints ครับ

นี่คือการสร้าง "ชนิดข้อมูลมาตรฐาน" ของป๋าเอง แล้วเอาไปใช้ซ้ำได้ทั่วโปรแกรมเลยครับ

In [ ]:
from typing import Annotated
from pydantic import BaseModel, StringConstraints

# --- [1] สร้าง "มาตรฐาน" ของป๋าขึ้นมาครั้งเดียว ---

# มาตรฐานที่ 1: ตัวหนังสือที่ต้องตัดช่องว่างหน้าหลังเสมอ
StrippedStr = Annotated[str, StringConstraints(strip_whitespace=True)]

# มาตรฐานที่ 2: ตัวหนังสือที่ต้องตัดช่องว่าง และเปลี่ยนเป็นตัวพิมพ์ใหญ่เสมอ (เช่น สกุลเงิน)
UpperStr = Annotated[str, StringConstraints(strip_whitespace=True, to_upper=True)]

# --- [2] นำไปใช้ใน Classes ต่างๆ (ไม่ต้องเขียน Validator ซ้ำแล้ว!) ---

class TrackingNumber(BaseModel):
    # ป๋าใช้ StrippedStr แทน str ได้เลย มันจะ strip ให้เองอัตโนมัติ!
    value: StrippedStr 

class Money(BaseModel):
    amount: float
    # ใช้ UpperStr ตัวเดียว จบทั้ง strip และ upper ในบรรทัดเดียว
    currency: UpperStr 

# --- ทดสอบผลลัพธ์ ---
t = TrackingNumber(value="  TH12345  ")
print(f"Tracking: '{t.value}'") # ผลลัพธ์: 'TH12345'

m = Money(amount=100, currency=" thb ")
print(f"Currency: '{m.currency}'") # ผลลัพธ์: 'THB'

จัดไปครับป๋า! นี่คือขั้นตอนการ Refactor (รี-แฟค-เตอร์-ริง) ครั้งใหญ่ เพื่อเปลี่ยนจากโค้ดที่ "ก๊อปวาง" มาเป็นโค้ดที่ "เป็นระบบ" โดยใช้หลักการ DRY (Don't Repeat Yourself) ครับ

เราจะนำ Annotated และ StringConstraints มาใช้สร้าง "มาตรฐานข้อมูล" ของป๋าเองครับ

📝 ส่วนที่ 1: การออกแบบโครงสร้างใหม่ (Refactoring Design)

# 🛠️ บทที่ 4.15: การรีแฟคเตอร์สู่ความเป็นมืออาชีพ (Refactoring to Professional Standards)
---

การรีแฟคเตอร์ไม่ใช่การเปลี่ยน "สิ่งที่โปรแกรมทำ" แต่เป็นการเปลี่ยน "วิธีที่โปรแกรมเขียน" ให้สะอาดและดูแลง่ายขึ้นครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Refactoring** | รี-แฟค-เตอร์-ริง | **การปรับปรุงโครงสร้างโค้ด** ให้ดีขึ้นโดยที่ผลลัพธ์การทำงานยังเหมือนเดิม |
| **Type Alias** | ไทป์ เอ-เลียส | **การตั้งชื่อเล่นให้ชนิดข้อมูล** (เช่น สร้าง StrippedStr มาใช้แทน str ปกติ) |
| **Encapsulation** | เอน-แคป-ซู-เล-ชัน | **การห่อหุ้ม** (การเก็บกฎการตัดช่องว่างไว้ที่เดียว ไม่กระจายไปทั่ว) |
| **Boilerplate Code** | บอย-เลอร์-เพลต โค้ด | **โค้ดซ้ำซาก** (โค้ดที่ต้องเขียนเหมือนเดิมซ้ำ ๆ ในหลายที่ ซึ่งเราจะกำจัดทิ้ง) |

**🔍 แผนการรีแฟคเตอร์ของป๋า:**
1. **กำจัด `@field_validator`:** ที่เคยเขียน `.strip()` หรือ `.upper()` ซ้ำ ๆ ในทุก Class
2. **สร้างมาตรฐานกลาง:** ใช้ `StrippedStr` สำหรับ ID/Tracking และ `UpperStr` สำหรับ Currency
3. **ใช้ Type Hint:** ให้ Pydantic จัดการงานสกปรก (การตัดช่องว่าง) ให้เราเองอัตโนมัติ

In [ ]:
from typing import Annotated, Optional
from pydantic import BaseModel, Field, StringConstraints, field_validator
import pandas as pd

# ==========================================
# 🎯 1. มาตรฐานข้อมูลกลาง (DRY - Type Aliases)
# ==========================================
# 1.1 ตัดเว้นวรรคให้อัตโนมัติ (ใช้แทน str ทั่วไป)
StrippedStr = Annotated[str, StringConstraints(strip_whitespace=True, min_length=1)]

# 1.2 ตัดเว้นวรรค + เป็นตัวพิมพ์ใหญ่เสมอ (สำหรับสกุลเงิน)
UpperStr = Annotated[str, StringConstraints(strip_whitespace=True, to_upper=True, min_length=3, max_length=3)]


# ==========================================
# 📦 2. Value Objects (ชิ้นส่วนพื้นฐาน)
# ==========================================
class TrackingNumber(BaseModel):
    value: StrippedStr # ใช้มาตรฐาน StrippedStr ได้เลย

    @field_validator('value')
    @classmethod
    def check_length(cls, v):
        if len(v) < 5:
            raise ValueError('Tracking Number สั้นเกินไป')
        return v # ไม่ต้อง .strip() แล้ว เพราะ StrippedStr ทำให้ก่อนแล้ว

class TicketId(BaseModel):
    value: StrippedStr # จบในบรรทัดเดียว ไม่ต้องมี @field_validator

class Money(BaseModel):
    amount: float = Field(..., ge=0)
    currency: UpperStr # จบในบรรทัดเดียว ได้ทั้ง strip และ upper

    def add(self, other: 'Money') -> 'Money':
        if self.currency != other.currency:
            raise ValueError(f'Cannot add different currencies: {self.currency} and {other.currency}')
        return Money(amount=self.amount + other.amount, currency=self.currency)

    def __str__(self):
        return f'{self.amount:,.2f} {self.currency}'


# ==========================================
# 🏛️ 3. Entities & Aggregates (โมเดลธุรกิจ)
# ==========================================
class ClaimTicket(BaseModel):
    """
    Entity: ใบเคลมสินค้า
    """
    ticket_id: TicketId
    tracking_number: TrackingNumber
    compensation_amount: Money
    version: int = Field(default=1)

    def update_compensation(self, new_money: Money):
        self.compensation_amount = new_money
        self.version += 1
        print(f'อัพเดทยอดเงินเป็น {new_money} (version: {self.version})')

class ClaimCase(BaseModel):
    """
    Aggregate Root: ตระกร้าคุมใบเคลมของสินค้า 1 ชิ้น
    """
    tracking_number: TrackingNumber
    tickets: list[ClaimTicket] = Field(default_factory=list)
    total_compensation: Money = Field(default_factory=lambda: Money(amount=0, currency="THB"))

    def add_ticket(self, ticket: ClaimTicket):
        if ticket.tracking_number.value != self.tracking_number.value:
            raise ValueError("ป๋าครับ! ใบเคลมนี้มันคนละเลข Tracking กันนะ")

        self.tickets.append(ticket)
        new_amount = self.total_compensation.amount + ticket.compensation_amount.amount
        self.total_compensation = Money(amount=new_amount, currency="THB")
        print(f"✅ เพิ่ม {ticket.ticket_id.value} สำเร็จ! ยอดรวมเคสนี้: {self.total_compensation}")


# ==========================================
# 🗄️ 4. Repositories (ลิ้นชักข้อมูล)
# ==========================================
class ClaimRepository:
    def save(self, claim_case: ClaimCase):
        raise NotImplementedError
        
    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        raise NotImplementedError

class InMemoryClaimRepository(ClaimRepository):
    def __init__(self):
        self._db = {} 

    def save(self, claim_case: ClaimCase):
        self._db[claim_case.tracking_number.value] = claim_case
        print(f"💾 [DB จำลอง] บันทึกเคสของพัสดุ {claim_case.tracking_number.value} ลงลิ้นชักเรียบร้อย!")

    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        found_case = self._db.get(tracking.value)
        if found_case:
            print(f"🔍 [DB จำลอง] ค้นพบเคสของพัสดุ {tracking.value}")
        else:
            print(f"❌ [DB จำลอง] ไม่พบพัสดุ {tracking.value} ในระบบ")
        return found_case

class PandasClaimRepository(ClaimRepository):
    def __init__(self, file_path: str):
        self.file_path = file_path

    def get_all_cases(self) -> list[ClaimCase]:
        df = pd.read_excel(self.file_path)
        
        # ปรับปรุง: ใช้ Dictionary เพื่อรวมกลุ่มใบเคลมที่ Tracking เดียวกันเข้าด้วยกัน 
        all_cases = {}
        
        for index, row in df.iterrows():
            tn = TrackingNumber(value=str(row['tracking_no']))
            tid = TicketId(value=str(row['complaint_ticket_id']))
            
            amount = row.get('compensation_final_amt', 0)
            if pd.isna(amount): amount = 0 # ดักจับกรณีเป็นค่าว่าง
            
            money = Money(amount=float(amount), currency="THB")

            ticket = ClaimTicket(
                ticket_id=tid, 
                tracking_number=tn, 
                compensation_amount=money
            )
            
            if tn.value not in all_cases:
                all_cases[tn.value] = ClaimCase(tracking_number=tn)
            
            all_cases[tn.value].add_ticket(ticket)
            
        return list(all_cases.values())


# ==========================================
# 🧠 5. Domain Services (ผู้ประสานงาน)
# ==========================================
class ClaimEnrichmentService:
    def enrich(self, claim_case: ClaimCase, compensation_map: dict[str, Money]):
        tracking_val = claim_case.tracking_number.value
        
        if tracking_val in compensation_map:
            real_money = compensation_map[tracking_val]
            for ticket in claim_case.tickets:
                ticket.compensation_amount = real_money
            
            new_amt = sum(t.compensation_amount.amount for t in claim_case.tickets)
            claim_case.total_compensation = Money(amount=new_amt, currency="THB")
            
            print(f"✨ เติมเงินให้ {tracking_val} สำเร็จ: {real_money}")
        else:
            print(f"⚠️ ไม่พบยอดเงินสำหรับ {tracking_val} ในข้อมูลไฟล์ 2")


# ==========================================
# 🧪 6. การรันใช้งานจริง (Main Execution)
# ==========================================
if __name__ == "__main__":
    print("=== ทดสอบ Value Objects ===")
    good_track = TrackingNumber(value='  TH1234567890  ') # ลองเว้นวรรค
    print(f'สำเร็จ: ได้รับพัสดุ {good_track.value}') # ผลลัพธ์ต้องไม่มีเว้นวรรค

    try:
        bad_track = TrackingNumber(value='TH12')
    except ValueError as e:
        print(f'โดนดัก error: {e}')

    try:
        claim_money = Money(amount=1477.50, currency=' thb ') # ลองเว้นวรรค + ตัวเล็ก
        print(f'สร้างสำเร็จ: {claim_money}') # ผลลัพธ์ต้องเป็นตัวใหญ่และไม่มีเว้นวรรค
        
        fake_money = Money(amount=-50, currency='usd')
    except Exception as e:
        print(f'ระบบดักจับเงินติดลบได้: {e}')

    print("\n=== ทดสอบ Entity & Aggregate ===")
    tn = TrackingNumber(value="TH1234567890")
    my_case = ClaimCase(tracking_number=tn)

    t1 = ClaimTicket(ticket_id=TicketId(value="CMP-01"), tracking_number=tn, compensation_amount=Money(amount=100, currency="THB"))
    my_case.add_ticket(t1)

    t2 = ClaimTicket(ticket_id=TicketId(value="CMP-02"), tracking_number=tn, compensation_amount=Money(amount=200, currency="THB"))
    my_case.add_ticket(t2)

    print("\n=== ทดสอบ Repository ===")
    repo = InMemoryClaimRepository()
    repo.save(my_case)
    result = repo.get_by_tracking(TrackingNumber(value="TH1234567890"))
    print(f"ยอดรวมในระบบตอนนี้: {result.total_compensation}")

# 🏗️ บทที่ 4.16: กฎของคลาสแม่และคลาสลูก (Interface & Implementation)
---

ในสถาปัตยกรรมซอฟต์แวร์ที่ดี (Clean Architecture) เราจะใช้ **"คลาสแม่"** เป็นตัวกำหนดสัญญา (Contract) ว่าระบบต้องการลิ้นชักแบบไหน และ **"คลาสลูก"** ที่ถูกสร้างขึ้นมาทีหลังจะต้องทำตามสัญญานั้นให้ครบถ้วน ห้ามขาดแม้แต่ข้อเดียว!



| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Interface / Contract** | อิน-เทอร์-เฟซ / คอน-แทรกต์ | **พิมพ์เขียว หรือ สัญญา** (คลาสแม่ที่กำหนดว่าลูกต้องมีคำสั่งอะไรบ้าง แต่ไม่เขียนวิธีทำไว้) |
| **Implementation** | อิม-พลี-เมน-เท-ชัน | **การลงมือทำจริง** (คลาสลูกที่รับพิมพ์เขียวไปสร้างจริง เช่น ไปต่อกับ RAM หรือ Pandas) |
| **Contract Violation** | คอน-แทรกต์ ไว-โอ-เล-ชัน | **การละเมิดสัญญา** (เมื่อคลาสลูกไม่มีคำสั่งตามที่คลาสแม่สั่งไว้ โปรแกรมจะเสี่ยงพัง) |
| **Polymorphism** | โพ-ลี-มอร์-ฟิซึม | **การพ้องรูป** (การเรียกใช้ `.save()` ได้เหมือนกัน ไม่ว่าลิ้นชักนั้นจะเป็น Pandas หรือ In-Memory) |

**🔍 ทำไม PandasClaimRepository ถึงทำผิดกฎ?**
ในตอนแรกคลาสแม่ `ClaimRepository` บังคับให้ลูกต้องมี `save` และ `get_by_tracking` แต่เราสร้าง `PandasClaimRepository` โดยมีแค่ `get_all_cases` ซึ่งถือว่า **ผิดสัญญา** ครับ!

---

### 💻 การแก้ไขโค้ดให้ถูกต้องตามหลัก OOP

เราต้องเพิ่ม `get_all_cases` ลงในคลาสแม่ (เพื่อให้เป็นมาตรฐาน) และบังคับให้คลาสลูกทุกตัวเขียนฟังก์ชันให้ครบถ้วนครับ

```python
from typing import Optional

# ---------------------------------------------------------
# 🏛️ 1. คลาสแม่ (Interface / สัญญาที่สมบูรณ์)
# ---------------------------------------------------------
class ClaimRepository:
    def save(self, claim_case: ClaimCase):
        raise NotImplementedError
        
    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        raise NotImplementedError

    def get_all_cases(self) -> list[ClaimCase]:
        raise NotImplementedError


# ---------------------------------------------------------
# 🚀 2. คลาสลูกคนที่ 1: InMemory (ทำงานบน RAM)
# ---------------------------------------------------------
class InMemoryClaimRepository(ClaimRepository):
    def __init__(self):
        self._db = {} 

    def save(self, claim_case: ClaimCase):
        self._db[claim_case.tracking_number.value] = claim_case

    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        return self._db.get(tracking.value)

    # ✅ ทำให้ครบสัญญา: เพิ่มการดึงทั้งหมด
    def get_all_cases(self) -> list[ClaimCase]:
        return list(self._db.values())


# ---------------------------------------------------------
# 🐼 3. คลาสลูกคนที่ 2: Pandas (ทำงานบนไฟล์ Excel)
# ---------------------------------------------------------
class PandasClaimRepository(ClaimRepository):
    def __init__(self, file_path: str):
        self.file_path = file_path

    # ✅ ทำตามสัญญาข้อที่ 1: ดึงข้อมูลทั้งหมด
    def get_all_cases(self) -> list[ClaimCase]:
        # (โค้ดอ่าน Excel ด้วย Pandas และจัดกลุ่ม ClaimCase ที่ป๋าเขียนไว้)
        pass 

    # ✅ ทำตามสัญญาข้อที่ 2: ค้นหาทีละตัว
    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        all_cases = self.get_all_cases()
        for case in all_cases:
            if case.tracking_number.value == tracking.value:
                return case
        return None

    # ✅ ทำตามสัญญาข้อที่ 3: การบันทึก
    def save(self, claim_case: ClaimCase):
        # ไฟล์ Excel การจะเซฟทีละบรรทัดมันยาก เราเลยดัก Error ไว้ก่อนเพื่อบอกให้คนอื่นรู้
        raise Exception("PandasClaimRepository ตอนนี้เป็นแบบ Read-Only ยังไม่รองรับการเซฟทีละเคสครับ!")

# 🏛️ บทที่ 4.17: กฎเหล็กของระบบ (Abstract Base Classes)
---

บอกลา "สัญญาใจ" แล้วก้าวเข้าสู่ "สัญญากฎหมาย" ครับ การใช้ `ABC` จะทำให้ Python ทิ้งนิสัยชิลๆ แล้วหันมาตรวจสอบคลาสลูกทุกตัวอย่างเข้มงวดตั้งแต่ตอนที่โปรแกรมเริ่มรัน (ไม่ต้องรอให้พังตอนเรียกใช้งาน)



| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Abstract Base Class (ABC)** | แอ็บ-สแตรก เบส คลาส | **คลาสแม่แบบนามธรรม** (คลาสที่สร้างขึ้นมาเพื่อเป็นพิมพ์เขียว ห้ามเอาไปใช้งานตรงๆ) |
| **Abstract Method** | แอ็บ-สแตรก เมท-ธอด | **ฟังก์ชันบังคับ** (ต้องมี `@abstractmethod` แปะไว้บนหัว เพื่อสั่งให้คลาสลูกต้องเขียนฟังก์ชันนี้เสมอ) |
| **Compile-time Error** | คอม-ไพล์-ไทม์ เออร์-เรอร์ | **การพังตั้งแต่ยังไม่เริ่ม** (ข้อดีของการใช้ ABC คือถาลูกเขียนไม่ครบ โปรแกรมจะแจ้งเตือนทันที ไม่ปล่อยให้หลุดไปพังหน้างาน) |

**🔍 สิ่งที่ป๋าจะได้จากการ Refactor รอบนี้:**
1. คลาส `ClaimRepository` จะกลายเป็นผู้คุมกฎหมาย
2. `PandasClaimRepository` จะต้องถูกบังคับให้เขียนปุ่ม `.save()` และ `.get_by_tracking()` (แม้ว่าข้างในจะเขียนดักไว้ว่า "ทำไม่ได้" ก็ตาม)

💻 ส่วนที่ 2: โค้ดฉบับ "สมบูรณ์แบบไร้รอยต่อ" (The Ultimate Refactored Code)
นี่คือร่างสมบูรณ์ที่สุดของ Domain Layer ครับ ป๋าก๊อปปี้ไปทับของเดิมได้เลย จุกทองรวมทั้งมาตรฐานตัวหนังสือ (Annotated) และมาตรฐานคลาสแม่ (ABC) ไว้ให้หมดแล้วครับ!

In [ ]:
from abc import ABC, abstractmethod
from typing import Annotated, Optional, List
from pydantic import BaseModel, Field, StringConstraints, field_validator
import pandas as pd

# ==========================================
# 🎯 1. มาตรฐานข้อมูลกลาง (DRY - Type Aliases)
# ==========================================
StrippedStr = Annotated[str, StringConstraints(strip_whitespace=True, min_length=1)]
UpperStr = Annotated[str, StringConstraints(strip_whitespace=True, to_upper=True, min_length=3, max_length=3)]


# ==========================================
# 📦 2. Value Objects
# ==========================================
class TrackingNumber(BaseModel):
    value: StrippedStr
    @field_validator('value')
    @classmethod
    def check_length(cls, v):
        if len(v) < 5: raise ValueError('Tracking Number สั้นเกินไป')
        return v

class TicketId(BaseModel):
    value: StrippedStr

class Money(BaseModel):
    amount: float = Field(..., ge=0)
    currency: UpperStr

    def add(self, other: 'Money') -> 'Money':
        if self.currency != other.currency:
            raise ValueError(f'Cannot add different currencies: {self.currency} and {other.currency}')
        return Money(amount=self.amount + other.amount, currency=self.currency)

    def __str__(self):
        return f'{self.amount:,.2f} {self.currency}'


# ==========================================
# 🏛️ 3. Entities & Aggregates
# ==========================================
class ClaimTicket(BaseModel):
    ticket_id: TicketId
    tracking_number: TrackingNumber
    compensation_amount: Money
    version: int = Field(default=1)

    def update_compensation(self, new_money: Money):
        self.compensation_amount = new_money
        self.version += 1

class ClaimCase(BaseModel):
    tracking_number: TrackingNumber
    tickets: list[ClaimTicket] = Field(default_factory=list)
    total_compensation: Money = Field(default_factory=lambda: Money(amount=0, currency="THB"))

    def add_ticket(self, ticket: ClaimTicket):
        if ticket.tracking_number.value != self.tracking_number.value:
            raise ValueError("ป๋าครับ! ใบเคลมคนละเลข Tracking กันนะ")
        self.tickets.append(ticket)
        new_amount = self.total_compensation.amount + ticket.compensation_amount.amount
        self.total_compensation = Money(amount=new_amount, currency="THB")


# ==========================================
# 🗄️ 4. Repositories (ใช้ ABC เข้มงวด 100%)
# ==========================================
class ClaimRepository(ABC):
    @abstractmethod
    def save(self, claim_case: ClaimCase):
        pass # เป็นแค่พิมพ์เขียว ไม่ต้องมีโค้ดการทำงาน
        
    @abstractmethod
    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        pass

    @abstractmethod
    def get_all_cases(self) -> List[ClaimCase]:
        pass

# --- ลูกคนที่ 1: InMemory ---
class InMemoryClaimRepository(ClaimRepository):
    def __init__(self):
        self._db = {} 

    def save(self, claim_case: ClaimCase):
        self._db[claim_case.tracking_number.value] = claim_case

    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        return self._db.get(tracking.value)

    def get_all_cases(self) -> List[ClaimCase]:
        return list(self._db.values())

# --- ลูกคนที่ 2: Pandas ---
class PandasClaimRepository(ClaimRepository):
    def __init__(self, file_path: str):
        self.file_path = file_path

    def get_all_cases(self) -> List[ClaimCase]:
        df = pd.read_excel(self.file_path)
        all_cases = {}
        
        for _, row in df.iterrows():
            tn = TrackingNumber(value=str(row['tracking_no']))
            tid = TicketId(value=str(row['complaint_ticket_id']))
            amount = row.get('compensation_final_amt', 0)
            if pd.isna(amount): amount = 0
            
            money = Money(amount=float(amount), currency="THB")
            ticket = ClaimTicket(ticket_id=tid, tracking_number=tn, compensation_amount=money)
            
            if tn.value not in all_cases:
                all_cases[tn.value] = ClaimCase(tracking_number=tn)
            all_cases[tn.value].add_ticket(ticket)
            
        return list(all_cases.values())

    def get_by_tracking(self, tracking: TrackingNumber) -> Optional[ClaimCase]:
        # ค้นหาจากเคสทั้งหมด
        all_cases = self.get_all_cases()
        for case in all_cases:
            if case.tracking_number.value == tracking.value:
                return case
        return None

    def save(self, claim_case: ClaimCase):
        # ดักไว้ชัดเจนว่า Pandas ทำงานแบบ Read-Only สำหรับตอนนี้
        raise NotImplementedError("PandasClaimRepository ออกแบบมาให้อ่านอย่างเดียวครับป๋า!")


# ==========================================
# 🧠 5. Domain Services
# ==========================================
class ClaimEnrichmentService:
    def enrich(self, claim_case: ClaimCase, compensation_map: dict[str, Money]):
        tracking_val = claim_case.tracking_number.value
        
        if tracking_val in compensation_map:
            real_money = compensation_map[tracking_val]
            for ticket in claim_case.tickets:
                ticket.compensation_amount = real_money
            
            new_amt = sum(t.compensation_amount.amount for t in claim_case.tickets)
            claim_case.total_compensation = Money(amount=new_amt, currency="THB")
            print(f"✨ เติมเงินให้ {tracking_val} สำเร็จ: {real_money}")


# ==========================================
# 🧪 6. การรันใช้งานจริง (Main Execution)
# ==========================================
if __name__ == "__main__":
    print("=== ทดสอบ Value Objects ===")
    good_track = TrackingNumber(value='  TH1234567890  ') # ลองเว้นวรรค
    print(f'สำเร็จ: ได้รับพัสดุ {good_track.value}') # ผลลัพธ์ต้องไม่มีเว้นวรรค

    try:
        bad_track = TrackingNumber(value='TH12')
    except ValueError as e:
        print(f'โดนดัก error: {e}')

    try:
        claim_money = Money(amount=1477.50, currency=' thb ') # ลองเว้นวรรค + ตัวเล็ก
        print(f'สร้างสำเร็จ: {claim_money}') # ผลลัพธ์ต้องเป็นตัวใหญ่และไม่มีเว้นวรรค
        
        fake_money = Money(amount=-50, currency='usd')
    except Exception as e:
        print(f'ระบบดักจับเงินติดลบได้: {e}')

    print("\n=== ทดสอบ Entity & Aggregate ===")
    tn = TrackingNumber(value="TH1234567890")
    my_case = ClaimCase(tracking_number=tn)

    t1 = ClaimTicket(ticket_id=TicketId(value="CMP-01"), tracking_number=tn, compensation_amount=Money(amount=100, currency="THB"))
    my_case.add_ticket(t1)

    t2 = ClaimTicket(ticket_id=TicketId(value="CMP-02"), tracking_number=tn, compensation_amount=Money(amount=200, currency="THB"))
    my_case.add_ticket(t2)

    print("\n=== ทดสอบ Repository ===")
    repo = InMemoryClaimRepository()
    repo.save(my_case)
    result = repo.get_by_tracking(TrackingNumber(value="TH1234567890"))
    print(f"ยอดรวมในระบบตอนนี้: {result.total_compensation}")

=== ทดสอบ Value Objects ===
สำเร็จ: ได้รับพัสดุ TH1234567890
โดนดัก error: 1 validation error for TrackingNumber
value
  Value error, Tracking Number สั้นเกินไป [type=value_error, input_value='TH12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
สร้างสำเร็จ: 1,477.50 THB
ระบบดักจับเงินติดลบได้: 1 validation error for Money
amount
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-50, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal

=== ทดสอบ Entity & Aggregate ===

=== ทดสอบ Repository ===
ยอดรวมในระบบตอนนี้: 300.00 THB


🧪 บทที่ 4.18: การทดสอบการเปลี่ยนแปลงสถานะ (State Change & Versioning)
หัวใจสำคัญของการเขียนโปรแกรมระดับสูงคือการทำให้ข้อมูล "มีความน่าเชื่อถือ" ครับ ในบทนี้เราจะทดสอบระบบ Optimistic Locking หรือการใช้ฟิลด์ version เพื่อติดตามว่าข้อมูลถูกแก้ไขไปกี่ครั้งแล้ว เพื่อป้องกันปัญหาข้อมูลชนกันเวลาทำงานจริงครับ

คำศัพท์ (Technical Term),คำอ่านไทย (Pronunciation),ความหมาย (Translation/Definition)
State Change,สเตท เชนจ์,การเปลี่ยนแปลงสถานะ เช่น การอัปเดตยอดเงินในใบเคลม
Optimistic Locking,ออพ-ติ-มิส-ติก ล็อก-กิ้ง,การล็อกแบบมองโลกในแง่ดี ใช้เลขเวอร์ชันเช็กว่าข้อมูลถูกใครแอบแก้ตัดหน้าไปก่อนไหม
In-Memory State,อิน-เมม-โม-รี สเตท,สถานะในหน่วยความจำ ข้อมูลปัจจุบันที่ยังไม่ได้เซฟลงลิ้นชัก
Invariant Requirement,อิน-แว-เรียนท์ รี-ไคว-เมนท์,ข้อกำหนดที่ไม่เปลี่ยนแปลง กฎที่ต้องเป็นจริงเสมอ (เช่น แก้เงินแล้วเวอร์ชันต้องขึ้น)

🔍 สิ่งที่ป๋าจะได้พิสูจน์ในเทสเคสที่ 20:

เมื่อสร้าง ClaimTicket ครั้งแรก เลข version ต้องเริ่มต้นที่ 1 เสมอ

เมื่อเรียกฟังก์ชัน update_compensation เลข version ต้องขยับขึ้นเป็น 2, 3, ... ตามลำดับ

ข้อมูลใน Entity ต้องมีความเป็นระเบียบและตรวจสอบย้อนกลับได้

In [ ]:
def test_claim_ticket_version_increments_on_update():
    """
    TDD Test 20: ทดสอบว่าเมื่อมีการแก้ไขข้อมูล (State Change) 
    เลข Version ของ Entity ต้องขยับขึ้นเสมอ
    """
    # 1. [Arrange] สร้างใบเคลมเวอร์ชันเริ่มต้น
    tn = TrackingNumber(value="TH-99999")
    ticket = ClaimTicket(
        ticket_id=TicketId(value="CMP-TEST-020"),
        tracking_number=tn,
        compensation_amount=Money(amount=100, currency="THB")
    )
    
    # ตรวจสอบค่าเริ่มต้น (Version ต้องเป็น 1)
    assert ticket.version == 1
    assert ticket.compensation_amount.amount == 100.0

    # 2. [Act] พนักงานทำการแก้ไขยอดเงินชดเชย
    new_money = Money(amount=500, currency="THB")
    ticket.update_compensation(new_money)

    # 3. [Assert] ยอดเงินต้องเปลี่ยน และเลข Version ต้องขยับเป็น 2
    assert ticket.compensation_amount.amount == 500.0
    assert ticket.version == 2
    
    # แก้ไขซ้ำอีกรอบเพื่อดูความต่อเนื่อง
    ticket.update_compensation(Money(amount=1000, currency="THB"))
    assert ticket.version == 3

    print(f"✅ Test 20 Passed: ระบบติดตามเวอร์ชันของ {ticket.ticket_id.value} ทำงานถูกต้อง!")

🏆 ความหมายของเลข 20 (Milestone)
การที่ป๋ามี 20 เทสเคสที่ผ่านฉลุย (Passed) หมายความว่า Domain Logic ของป๋าครอบคลุมทุกมิติแล้วครับ:

✅ Data Validation: ข้อมูลขยะเข้าไม่ได้

✅ Business Logic: คำนวณเงินถูกต้อง

✅ Aggregation: รวมเคสซ้ำได้แม่นยำ

✅ Architecture: มี ABC คุมกฎโครงสร้าง

✅ State Management: มีระบบ Versioning รองรับการแก้ไข

# 📔 บทสรุปบทที่ 4: การสร้าง "สมองกล" (Domain Layer)
---

ในบทนี้ ป๋าได้สร้างรากฐานที่สำคัญที่สุดของระบบด้วยหลักการ **DDD (Domain-Driven Design)** และ **Clean Architecture** โดยเน้นที่ความถูกต้องแม่นยำและการตรวจสอบได้ 100% ครับ



### 1. การสร้างชิ้นส่วนที่เชื่อถือได้ (Value Objects)
เราเลิกใช้ข้อมูลดิบ (Raw Data) แต่หันมาใช้ **Value Objects** เพื่อป้องกันข้อมูลขยะ (Garbage In) ตั้งแต่หน้าประตูบ้านครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Value Object** | แวล-ลู ออบ-เจกต์ | **วัตถุที่มีค่าในตัวเอง** (เช่น เงิน, เลขพัสดุ) ถ้าค่าเปลี่ยน คือตัวตนเปลี่ยน |
| **Validation** | แวล-ลิ-เด-ชัน | **การตรวจสอบความถูกต้อง** (เช่น ยอดเงินต้องไม่ติดลบ) |
| **DRY Principle** | ดราย พริน-ซิ-เพิล | **Don't Repeat Yourself** (การไม่เขียนโค้ดซ้ำซาก โดยใช้ Type Alias) |

---

### 2. โครงสร้างและการคุมกฎ (Entity & Aggregate)
เราออกแบบให้ระบบมีการจัดการสถานะและการคำนวณที่ "รวมศูนย์" อยู่ในที่เดียว

* **ClaimTicket (Entity):** ใบแจ้งเคลมที่มี "ตัวตน" (Identity) และมีระบบ **Optimistic Locking (version)** เพื่อติดตามการแก้ไขและป้องกันการเซฟทับกัน
* **ClaimCase (Aggregate Root):** ตระกร้าใบใหญ่ที่คุมกฎเหล็กว่า "ห้ามเอาใบแจ้งคนละเลข Tracking มาใส่รวมกัน" และเป็นคนคำนวณยอดเงินรวมที่ถูกต้องที่สุด



---

### 3. พิมพ์เขียวและสัญญากฎหมาย (Repository & ABC)
เรานำ **ABC (Abstract Base Class)** มาใช้บังคับมาตรฐานการเชื่อมต่อข้อมูล เพื่อให้โปรแกรมขยายตัวได้ง่ายในอนาคต

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Interface / ABC** | อิน-เทอร์-เฟซ / เอ-บี-ซี | **สัญญากฎหมาย** ที่บังคับว่าลิ้นชักข้อมูล (Repository) ทุกอันต้องทำงานได้เหมือนกัน |
| **Implementation** | อิม-พลี-เมน-เท-ชัน | **การลงมือทำจริง** เช่น การเขียนให้ลิ้นชักไปอ่านจาก Pandas หรือ InMemory |
| **Persistence Ignorance** | เพอร์-ซิส-เทนซ์ อิก-โน-แรนซ์ | **การไม่ยึดติดกับฐานข้อมูล** สมองกลทำงานได้โดยไม่ต้องสนว่าไฟล์จริงจะเป็น Excel หรือ SQL |

---

### 4. เกราะป้องกัน 20 ชั้น (TDD - Test Driven Development)
ป๋าได้สร้าง Unit Test ทั้งหมด 20 เคส เพื่อรับประกันว่าไม่ว่าจะเจอสถานการณ์ไหน โปรแกรมของป๋าต้องไม่พัง (Fault Tolerance)

| ประเภทการทดสอบ | สิ่งที่ป๋าพิสูจน์แล้ว |
| :--- | :--- |
| **Happy Path** | ข้อมูลมาครบถ้วน คำนวณเงินถูกต้อง |
| **Sad Path** | เลข Tracking ผิด หรือสกุลเงินไม่ตรง ระบบต้องดัก Error ได้ |
| **Edge Case** | ยอดเงิน 0 บาท หรือมีช่องว่าง (Space) นัวเนีย ระบบต้องทำความสะอาดข้อมูลได้เอง |
| **Corner Case** | การแจ้งเคลมซ้ำหลายครั้งใน 1 Tracking ระบบต้องรวมยอดได้ไม่ตกหล่น |

---

> **💡 ข้อคิดจากเดฟจุกทอง:**
> "การเขียนโปรแกรมให้อ่านยากนั้นง่าย แต่การเขียนให้อ่านง่ายและตรวจสอบได้นั้นยากที่สุด... วันนี้ป๋าทำสำเร็จแล้วครับ!"

---

# 🤝 บทที่ 4.99: เทคนิคการคุยกับ Domain Expert (สะพานเชื่อมธุรกิจสู่โค้ด)
---

ก่อนที่ป๋าจะเริ่มเขียนโค้ดในบทที่ 5 การทำความเข้าใจ "ความต้องการที่แท้จริง" คือหัวใจสำคัญครับ ในทาง DDD เราไม่ได้แค่รับ Requirement มาทำตาม แต่เราต้องสร้างภาษาร่วมกันขึ้นมาครับ



| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Domain Expert** | โด-เมน เอ็กซ์-เพิร์ท | **ผู้เชี่ยวชาญในธุรกิจ** คนที่รู้ขั้นตอนการทำงานจริง (เช่น พนักงานเคลมสินค้า) |
| **Ubiquitous Language** | ยู-บิ-ควิ-ตัส แลง-เกวจ | **ภาษาสากลในระบบ** ภาษาที่ทั้งคนเขียนโค้ดและคนทำงานใช้เรียกสิ่งเดียวกัน (เช่นคำว่า "Ticket", "Case") |
| **Bounded Context** | บาวน์-เดด คอน-เทกซ์ | **ขอบเขตของงาน** การระบุว่าโปรแกรมเราจะทำแค่ไหน และจะไม่ทำอะไรบ้าง |
| **User Story** | ยู-เซอร์ สตอ-รี | **เรื่องราวของผู้ใช้** การเล่าว่า "ใคร" ทำ "อะไร" และ "ทำไม" |

---

### 💡 3 คำถามเด็ดที่ป๋าควรเอาไปถาม Domain Expert พรุ่งนี้

เพื่อให้ป๋าสกัดเอา Business Logic มาใส่ใน Domain Layer ได้แม่นยำ ป๋าลองถามเขาแบบนี้ครับ:

1. **"กฎเหล็กที่ห้ามพลาดคืออะไร?" (The Golden Rule)**
   - *คำถาม:* "ยอดเงินชดเชยจะถูกตัดสินจากอะไร และมีกรณีไหนไหมที่ห้ามจ่ายเงินเด็ดขาด?"
   - *ประโยชน์:* ป๋าจะได้เอามาเขียนใน `Money` หรือ `ClaimCase` เพื่อดัก Error (Validation)

2. **"ถ้าข้อมูลมาไม่ครบ พี่ทำยังไงต่อ?" (The Sad Path)**
   - *คำถาม:* "ถ้าในไฟล์ยอดเงิน (ไฟล์ 2) ไม่มีเลข Tracking ที่แจ้งเคลม พี่จะปล่อยทิ้งไว้ หรือต้องรีบแจ้งใครไหม?"
   - *ประโยชน์:* ป๋าจะได้รู้ว่าควรเขียน Code ดัก `if-else` หรือส่ง Log แจ้งเตือนยังไง

3. **"สิ่งที่พี่ปวดหัวที่สุดเวลาทำงานมือ (Manual) คืออะไร?" (The Pain Point)**
   - *คำถาม:* "ปกติที่พี่ทำมือ พี่กลัวพลาดตรงไหนที่สุด?"
   - *ประโยชน์:* นี่คือจุดที่ป๋าต้องเขียน **Test Case** มาดักไว้ให้เขาครับ เช่น การบวกเงินผิด หรือการมองข้ามบรรทัดที่ซ้ำกัน

---

### 🛠️ ตัวอย่างการสร้าง "ภาษาสมมติ" (Ubiquitous Language)

ในการคุยกัน พยายามตกลงคำเรียกให้ตรงกันครับ เช่น:
- **ป๋า:** "พี่ครับ คำว่า 'ใบแจ้ง' กับ 'คำร้อง' คืออันเดียวกันไหม?"
- **Expert:** "อ๋อ อันเดียวกันจ้ะ"
- **ป๋า:** "งั้นในโปรแกรม ผมขอเรียกมันว่า **'ClaimTicket'** อย่างเดียวเลยนะพี่ จะได้เข้าใจตรงกัน"

> **💡 ข้อคิดจากจุกทอง:**
> "ป๋าไม่ต้องรีบเขียนโค้ดให้เสร็จครับ แต่ต้องรีบคุยให้เคลียร์... โค้ดที่เขียนบนความเข้าใจผิด คือโค้ดที่ต้องลบทิ้งในวันหน้าครับ!"

---

# 🤝 บทที่ 4.99: ชุดคำถามสำหรับ Domain Expert (สกัดลอจิกประกอบร่าง 2 ไฟล์)
---

ก่อนจะเริ่มเขียนโค้ดประกอบร่างข้อมูลในบทที่ 5 ป๋าควรใช้ชุดคำถามเหล่านี้เพื่อสร้าง **Ubiquitous Language** และทำความเข้าใจกฎธุรกิจที่ซ่อนอยู่จากผู้เชี่ยวชาญครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Primary Key / Linker** | ไพ-แม-รี คีย์ / ลิงก์-เกอร์ | **ตัวเชื่อมข้อมูล** (เช่น Tracking Number ที่ใช้เชื่อมไฟล์ 1 และไฟล์ 2) |
| **Data Discrepancy** | ดาต้า ดิส-เคร-แพน-ซี | **ความไม่สอดคล้องของข้อมูล** (เช่น ไฟล์หนึ่งมีเลขนี้ แต่อีกไฟล์ไม่มี) |
| **Logic Validation** | ลอ-จิก แวล-ลิ-เด-ชัน | **การตรวจสอบความถูกต้องตามกฎ** (เงื่อนไขที่ตัดสินว่าข้อมูลนี้ใช้ได้หรือไม่) |
| **Rounding Policy** | ราวน์-ดิ้ง พอ-ลิ-ซี | **นโยบายการปัดเศษ** (กฎการจัดการทศนิยมของยอดเงินชดเชย) |

---

### ❓ 1. คำถามเรื่องการจับคู่ข้อมูล (Data Mapping)
* "พี่ครับ ในไฟล์แจ้งเคลม (ไฟล์ 1) และไฟล์ยอดเงิน (ไฟล์ 2) **เลข Tracking** คือตัวเดียวที่ใช้จับคู่กันใช่ไหมครับ? มีกรณีที่เลข Tracking เดียวกันแต่เป็นคนละรายการไหม?"
* "ในไฟล์ยอดเงิน ถ้ามีเลข Tracking ซ้ำกันหลายบรรทัด ผมควรจะ **'รวมยอดทั้งหมด'** หรือ **'เลือกบรรทัดล่าสุด'** ครับ?"

### ❓ 2. คำถามเรื่องความถูกต้อง (Validation & Rules)
* "สกุลเงินในไฟล์ยอดเงิน **เป็น THB (บาท) เสมอไหมครับ?** มีโอกาสที่จะเจอสกุลเงินอื่นปนมาไหม?"
* "ยอดเงินชดเชยมี **เพดานสูงสุด** ไหมครับ? เช่น ห้ามเกินยอดที่แจ้งเคลมมา หรือห้ามเกิน 50,000 บาท?"

### ❓ 3. คำถามเรื่องกรณีข้อมูลผิดปกติ (Sad Path & Edge Case)
* "ถ้าผมเจอ Tracking ในไฟล์แจ้งเคลม **แต่หาไม่เจอในไฟล์ยอดเงิน** พี่อยากให้โปรแกรมสรุปออกมาว่าอย่างไรครับ? (เช่น ให้เป็น 0 บาท หรือให้ข้ามไปเลย?)"
* "ถ้าตัวเลขในไฟล์ยอดเงินติดลบ หรือเป็น 0 บาท พี่ถือว่าปกติไหมครับ?"

### ❓ 4. คำถามเรื่องผลลัพธ์ที่ต้องการ (Final Output)
* "หลังจากรวมข้อมูลเสร็จแล้ว พี่อยากได้ไฟล์สรุปที่มีคอลัมน์ไหนบ้างครับ? และอยากให้ **เรียงลำดับ** ตามอะไรเป็นหลัก (เช่น เรียงตามวันที่ หรือเลข Ticket?)"

---

> **💡 ข้อคิดจากจุกทอง:**
> "ป๋าอย่าลืมจดคำตอบที่ได้มาเป็นข้อๆ นะครับ เพราะคำตอบเหล่านั้นจะกลายเป็น **Test Case** ที่เราจะเอามาเขียนดักไว้ใน Domain Layer เพื่อให้โปรแกรมของป๋าแม่นยำที่สุดครับ!"

---